! alle Pakete installieren

# Pakete importieren 

In [ ]:
import random
import statistics

# Code

## 1) Parameter des Systems

In [ ]:
LAMBDA_ANKUNFT_PRO_WOCHE = 20   # Ø neue Anmeldungen pro Woche
SIMULATIONS_DAUER_WOCHEN = 200  # über wie viele Wochen simuliert wird

# Kapazität
ANZAHL_THERAPEUTEN = 5          # Anzahl Therapeut*innen

# Therapiedauer (Servicezeit) in Wochen
MIN_THERAPIEDAUER = 10          # z. B. 10 Sitzungen à 1 Woche
MAX_THERAPIEDAUER = 40          # z. B. 40 Sitzungen

# Zufalls-Seed für Reproduzierbarkeit (ändern(?))
random.seed(42)

## 2) Funktionen

In [ ]:
def generiere_ankunftszeiten(lambda_pro_woche, sim_dauer_wochen):
    """
    Erzeugt eine Liste aufsteigender Ankunftszeiten (in Wochen)
    auf Basis eines Poisson-Prozesses (exponentielle Zwischenankunftszeiten).
    """
    ankunftszeiten = []
    aktuelle_zeit = 0.0

    # mittlere Zwischenankunftszeit
    mean_interarrival = 1.0 / lambda_pro_woche  # Wochen

    while aktuelle_zeit < sim_dauer_wochen:
        # Exponentiell verteilte Zwischenankunftszeit
        delta = random.expovariate(1.0 / mean_interarrival)
        aktuelle_zeit += delta
        if aktuelle_zeit > sim_dauer_wochen:
            break
        ankunftszeiten.append(aktuelle_zeit)

    return ankunftszeiten


def ziehe_therapiedauer(min_wochen, max_wochen):
    """
    Ziehe eine zufällige Therapiedauer (in Wochen).
    Hier: einfache Gleichverteilung zwischen min und max.
    Später kannst du z.B. Normal- oder andere Verteilungen verwenden.
    """
    return random.uniform(min_wochen, max_wochen)


## 3) Zielgrößen

## 4) Simulationslauf

In [ ]:
def simuliere_warteschlange():
    # 3.1 Ankunftszeiten generieren
    ankunftszeiten = generiere_ankunftszeiten(
        LAMBDA_ANKUNFT_PRO_WOCHE,
        SIMULATIONS_DAUER_WOCHEN
    )

    # nächster Zeitpunkt, an dem jede/r Therapeut*in frei ist
    # Index: Therapeut, Wert: Zeit (in Wochen)
    naechst_frei = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    # Listen zur Auswertung
    wartezeiten = []     # nur echte Wartezeit (bis Therapiebeginn)
    systemzeiten = []    # Zeit von Anmeldung bis Therapieende

    # zur Auslastungsberechnung: gesamte Behandlungszeit je Therapeut
    gesamt_behandlungszeit_pro_therapeut = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    for ankunft in ankunftszeiten:
        # Therapiedauer für diesen Patienten
        servicezeit = ziehe_therapiedauer(
            MIN_THERAPIEDAUER, MAX_THERAPIEDAUER
        )

        # 3.2 Finde den Therapeuten, der am frühesten frei ist
        therapeut_index = min(range(ANZAHL_THERAPEUTEN),
                              key=lambda i: naechst_frei[i])
        frueheste_frei_zeit = naechst_frei[therapeut_index]

        # 3.3 Therapiebeginn: max(Ankunft, frühester Frei-Zeitpunkt)
        therapiebeginn = max(ankunft, frueheste_frei_zeit)
        therapieende = therapiebeginn + servicezeit

        # 3.4 Warte- und Systemzeit berechnen
        wartezeit = therapiebeginn - ankunft
        systemzeit = therapieende - ankunft

        wartezeiten.append(wartezeit)
        systemzeiten.append(systemzeit)

        # 3.5 Kalender des Therapeuten aktualisieren
        naechst_frei[therapeut_index] = therapieende
        gesamt_behandlungszeit_pro_therapeut[therapeut_index] += servicezeit

    # ============================================
    # 4. Kennzahlen berechnen
    # ============================================

    # Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 5) Kennzahlen berechnen 

In [ ]:
# Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 6) Ergbenisse ausgeben

In [ ]:
print("=== Ergebnisse der Simulation (Therapie-Warteschlange) ===")
    print(f"Anzahl Therapeut*innen:            {ANZAHL_THERAPEUTEN}")
    print(f"Simulierte Zeit:                   {SIMULATIONS_DAUER_WOCHEN:.1f} Wochen")
    print(f"Anzahl Anmeldungen:                {len(ankunftszeiten)}")

    print("\nWartezeiten (in Wochen)")
    print(f"  Mittelwert:                      {mittlere_wartezeit:.2f}")
    print(f"  Median:                          {median_warte:.2f}")
    print(f"  75%-Quantil:                     {q75_warte:.2f}")
    print(f"  90%-Quantil:                     {q90_warte:.2f}")
    print(f"  Anteil > 12 Wochen:              {anteil_ueber_12*100:.1f} %")
    print(f"  Anteil > 24 Wochen:              {anteil_ueber_24*100:.1f} %")

    print("\nGesamtzeit im System (Anmeldung bis Therapieende)")
    print(f"  Mittelwert:                      {mittlere_systemzeit:.2f} Wochen")

    print("\nAuslastung der Therapeut*innen")
    for i, rho in enumerate(auslastungen):
        print(f"  Therapeut {i+1}:                 {rho*100:.1f} %")
    print(f"  Durchschnittliche Auslastung:    {mittlere_auslastung*100:.1f} %")
    
    if __name__ == "__main__":
    simuliere_warteschlange()

# Verbesserungen

=== Modell: Therapie (1 Sitzung/Woche, 24 Wochen), Kapazität = Sitzungen/Woche ===
Therapeut*innen:                14,600
Sitzungen/Woche pro Therapeut:  35
Kapazität (aktive Patient*innen): 511,000
Ankunftsrate:                   20,000 / Woche
Simulierte Ankünfte:            5,000,000
Warm-up ignoriert:              50,000
Reservoir (Quantile):           200,000

--- Run seed=42 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quantiles wait:         Q75=0.000, Q90=0.000, Q95=0.000
Share wait > 12 weeks:  0.00%
Share wait > 24 weeks:  0.00%
Mean system time:       24.000 weeks (SE=0.0000)

--- Run seed=43 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quantiles wait:         Q75=0.000, Q90=0.000, Q95=0.000
Share wait > 12 weeks:  0.00%
Share wait > 24 weeks:  0.00%
Mean system time:       24.000 weeks (SE=0.0000)

--- Run seed=44 (N=4,950,000) ---
Mean wait (weeks):      0.000  (SE=0.0000, CI95=[0.000, 0.000])
Quanti

## a)

In [2]:
# pip install simpy numpy pandas

import simpy
import numpy as np
import pandas as pd

# -----------------------------
# Parameter (aus Screenshot + sinnvolle Defaults)
# -----------------------------
YEAR_WEEKS = 52
SEED = 42
rng = np.random.default_rng(SEED)

TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557  # "Ja: 84.557 (88,82%)"
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR

N_THERAPISTS = 9000

THERAPY_SESSIONS = 24
SESSION_FREQUENCY_PER_WEEK = 1  # 1 Sitzung pro Woche
THERAPY_DURATION_WEEKS = THERAPY_SESSIONS / SESSION_FREQUENCY_PER_WEEK  # 24 Wochen

# Ankunftsprozess: Rate pro Woche wächst exponentiell: lambda(t) = lambda0 * exp(growth * t)
# - lambda0 wird so gewählt, dass im Mittel TOTAL_PATIENTS_YEAR im Jahr ankommen.
# - growth setzt die Stärke des Wachstums (0 = konstant)
ARRIVAL_GROWTH_PER_WEEK = 0.02  # <-- anpassen: 2% Wachstum pro Woche ist stark, nur Default!

# -----------------------------
# Hilfsfunktionen
# -----------------------------
def integral_exp_rate(lambda0, g, T):
    """Integral_0^T lambda0*exp(g t) dt"""
    if abs(g) < 1e-12:
        return lambda0 * T
    return lambda0 * (np.exp(g * T) - 1.0) / g

def solve_lambda0_for_total(total_expected, g, T):
    """Wähle lambda0 so, dass E[Ankünfte in (0,T)] = total_expected."""
    if abs(g) < 1e-12:
        return total_expected / T
    return total_expected * g / (np.exp(g * T) - 1.0)

LAMBDA0_PER_WEEK = solve_lambda0_for_total(TOTAL_PATIENTS_YEAR, ARRIVAL_GROWTH_PER_WEEK, YEAR_WEEKS)

def current_arrival_rate(env_now_week):
    return LAMBDA0_PER_WEEK * np.exp(ARRIVAL_GROWTH_PER_WEEK * env_now_week)

# Thinning für nicht-homogenen Poisson-Prozess
def nhpp_arrivals(env, out_store, T_end, rng):
    """
    Erzeuge Ankunftszeiten bis T_end mit Thinning.
    out_store bekommt Events (arrival_time).
    """
    t = 0.0
    # obere Schranke der Rate im Zeitintervall: bei exponentiellem Wachstum max am Ende
    lambda_max = current_arrival_rate(T_end)

    while True:
        # Kandidaten-Interarrival für homog. Poisson mit Rate lambda_max
        dt = rng.exponential(scale=1.0 / lambda_max)
        t += dt
        if t > T_end:
            break

        # Akzeptanzwahrscheinlichkeit
        lam_t = current_arrival_rate(t)
        if rng.random() < (lam_t / lambda_max):
            yield out_store.put(t)

def patient(env, pid, therapists, results, rng):
    """
    Patient kommt an, entscheidet ggf. ob er/sie wirklich startet, wartet auf Therapeut*in,
    belegt Therapeut*in für THERAPY_DURATION_WEEKS.
    """
    arrival = env.now

    # "Ja/Nein"-Gate (aus Screenshot)
    if rng.random() > P_START_THERAPY:
        results["dropped"].append(1)
        return
    results["dropped"].append(0)

    with therapists.request() as req:
        yield req
        start = env.now
        wait = start - arrival

        # Therapie dauert 24 Wochen (Default)
        yield env.timeout(THERAPY_DURATION_WEEKS)
        end = env.now

    results["wait_weeks"].append(wait)
    results["sojourn_weeks"].append(end - arrival)  # Warten + Therapie
    results["start_week"].append(start)
    results["arrival_week"].append(arrival)

def system(env, therapists, arrival_times_store, T_end, results, rng, queue_trace_every=1.0):
    """
    Startet Patient*innen bei ihren Ankunftszeiten und zeichnet Queue/Utilization grob auf.
    """
    pid = 0

    # Monitoring-Prozess
    def monitor():
        while env.now < T_end:
            # queue length = Anzahl wartender Requests
            q_len = len(therapists.queue)
            in_service = therapists.count
            results["queue_len_trace"].append((env.now, q_len))
            results["in_service_trace"].append((env.now, in_service))
            yield env.timeout(queue_trace_every)

    env.process(monitor())

    while True:
        t = yield arrival_times_store.get()
        if t > T_end:
            break
        # springe zur Ankunftszeit (falls wir "vorne" sind, timeout)
        if env.now < t:
            yield env.timeout(t - env.now)

        pid += 1
        env.process(patient(env, pid, therapists, results, rng))

# -----------------------------
# Run
# -----------------------------
def run_simulation():
    env = simpy.Environment()
    therapists = simpy.Resource(env, capacity=N_THERAPISTS)

    arrival_times_store = simpy.Store(env)
    results = {
        "wait_weeks": [],
        "sojourn_weeks": [],
        "arrival_week": [],
        "start_week": [],
        "dropped": [],
        "queue_len_trace": [],
        "in_service_trace": [],
    }

    env.process(nhpp_arrivals(env, arrival_times_store, YEAR_WEEKS, rng))
    env.process(system(env, therapists, arrival_times_store, YEAR_WEEKS, results, rng))

    env.run(until=YEAR_WEEKS + THERAPY_DURATION_WEEKS + 1)  # damit begonnene Therapien zu Ende laufen können

    # -----------------------------
    # Auswertung
    # -----------------------------
    df = pd.DataFrame({
        "arrival_week": results["arrival_week"],
        "start_week": results["start_week"],
        "wait_weeks": results["wait_weeks"],
        "sojourn_weeks": results["sojourn_weeks"],
    })

    dropped_rate = np.mean(results["dropped"])  # 1 = dropped
    started = len(df)

    # Monitoring
    q_trace = pd.DataFrame(results["queue_len_trace"], columns=["week", "queue_len"])
    s_trace = pd.DataFrame(results["in_service_trace"], columns=["week", "in_service"])
    utilization_est = (s_trace["in_service"].mean() / N_THERAPISTS) if len(s_trace) else np.nan

    summary = {
        "expected_arrivals_year": TOTAL_PATIENTS_YEAR,
        "lambda0_per_week": LAMBDA0_PER_WEEK,
        "growth_per_week": ARRIVAL_GROWTH_PER_WEEK,
        "therapists": N_THERAPISTS,
        "therapy_duration_weeks": THERAPY_DURATION_WEEKS,
        "started_patients": started,
        "dropped_share": dropped_rate,
        "mean_wait_weeks": float(df["wait_weeks"].mean()) if started else np.nan,
        "p90_wait_weeks": float(df["wait_weeks"].quantile(0.90)) if started else np.nan,
        "p95_wait_weeks": float(df["wait_weeks"].quantile(0.95)) if started else np.nan,
        "max_wait_weeks": float(df["wait_weeks"].max()) if started else np.nan,
        "mean_queue_len": float(q_trace["queue_len"].mean()) if len(q_trace) else np.nan,
        "max_queue_len": int(q_trace["queue_len"].max()) if len(q_trace) else None,
        "utilization_mean": float(utilization_est),
    }

    return summary, df, q_trace, s_trace

if __name__ == "__main__":
    summary, df, q_trace, s_trace = run_simulation()
    print("=== SUMMARY ===")
    for k, v in summary.items():
        print(f"{k}: {v}")

    print("\nBeispiel (erste 10 gestartete Patient*innen):")
    print(df.head(10))

=== SUMMARY ===
expected_arrivals_year: 95201
lambda0_per_week: 1040.893445152519
growth_per_week: 0.02
therapists: 9000
therapy_duration_weeks: 24.0
started_patients: 22861
dropped_share: 0.11090457194176988
mean_wait_weeks: 13.0896265898785
p90_wait_weeks: 32.28153640143486
p95_wait_weeks: 32.61657931976839
max_wait_weeks: 32.89300809477673
mean_queue_len: 22138.076923076922
max_queue_len: 61604
utilization_mean: 0.9023119658119658

Beispiel (erste 10 gestartete Patient*innen):
   arrival_week  start_week  wait_weeks  sojourn_weeks
0      0.003191    0.003191         0.0           24.0
1      0.003222    0.003222         0.0           24.0
2      0.004799    0.004799         0.0           24.0
3      0.004953    0.004953         0.0           24.0
4      0.005543    0.005543         0.0           24.0
5      0.006779    0.006779         0.0           24.0
6      0.007545    0.007545         0.0           24.0
7      0.007654    0.007654         0.0           24.0
8      0.007774    0

In [3]:
import random
from dataclasses import dataclass, field
from typing import List, Dict, Tuple, Optional

# -----------------------------
# VT Kapazität (deine Daten)
# -----------------------------
THERAPISTS_TOTAL = 9000

THERAPISTS_PART = 5490   # 61%
THERAPISTS_FULL = 3510   # 39%

SESSIONS_PER_WEEK_PART = 9    # ~20 Std
SESSIONS_PER_WEEK_FULL = 19   # ~40 Std

SLOTS_PER_WEEK = THERAPISTS_PART * SESSIONS_PER_WEEK_PART + THERAPISTS_FULL * SESSIONS_PER_WEEK_FULL
# = 116_190

# Therapie-Logik
THERAPY_SESSIONS = 24
SESSIONS_PER_PATIENT_PER_WEEK = 1
THERAPY_DURATION_WEEKS = THERAPY_SESSIONS // SESSIONS_PER_PATIENT_PER_WEEK  # 24 Wochen

# -----------------------------
# Arrival Modell
# -----------------------------
@dataclass
class ArrivalConfig:
    mode: str = "poisson"  # "poisson" oder "uniform"

    # Poisson: arrivals ~ Poisson(mean)
    poisson_mean: float = 1000.0

    # Uniform: arrivals ~ randint(low, high)
    uniform_low: int = 800
    uniform_high: int = 1200

# -----------------------------
# Simulation
# -----------------------------
@dataclass
class SimConfig:
    weeks: int = 52
    seed: int = 42
    slots_per_week: int = SLOTS_PER_WEEK
    therapy_duration_weeks: int = THERAPY_DURATION_WEEKS
    arrivals: ArrivalConfig = field(default_factory=ArrivalConfig)

@dataclass
class SimResults:
    started: int
    arrived: int
    still_waiting_end: int
    mean_wait_weeks: float
    p90_wait_weeks: float
    max_wait_weeks: int
    mean_queue_len: float
    max_queue_len: int
    mean_utilization: float
    trace_queue: List[Tuple[int, int]]
    trace_util: List[Tuple[int, float]]

def poisson(lmbda: float) -> int:
    """Simple Poisson sampler (Knuth). OK für moderate lmbda; für sehr große lmbda besser numpy."""
    import math
    L = math.exp(-lmbda)
    k = 0
    p = 1.0
    while p > L:
        k += 1
        p *= random.random()
    return k - 1

def sample_arrivals(week: int, cfg: ArrivalConfig) -> int:
    if cfg.mode == "poisson":
        return max(0, poisson(cfg.poisson_mean))
    elif cfg.mode == "uniform":
        return random.randint(cfg.uniform_low, cfg.uniform_high)
    else:
        raise ValueError("arrivals.mode muss 'poisson' oder 'uniform' sein")

def run_sim(cfg: SimConfig) -> SimResults:
    random.seed(cfg.seed)

    # Queue speichert Ankunftswochen (für Wartedauer)
    queue: List[int] = []

    # Active therapies: Liste verbleibender Wochen (jede belegt 1 Slot/Woche)
    active: List[int] = []

    wait_times: List[int] = []
    trace_queue: List[Tuple[int, int]] = []
    trace_util: List[Tuple[int, float]] = []

    total_arrived = 0
    total_started = 0
    queue_len_sum = 0
    util_sum = 0.0
    util_count = 0

    # Simulieren über "weeks + therapy_duration" damit wir Auslastung/Abbau sehen können
    total_weeks_sim = cfg.weeks + cfg.therapy_duration_weeks

    for w in range(total_weeks_sim):
        # 1) Therapien eine Woche "ablaufen" lassen
        if active:
            active = [rem - 1 for rem in active]
            active = [rem for rem in active if rem > 0]

        # 2) Neue Arrivals nur im Beobachtungsjahr (erste cfg.weeks Wochen)
        if w < cfg.weeks:
            a = sample_arrivals(w, cfg.arrivals)
            total_arrived += a
            # alle kommen in die Warteschlange
            queue.extend([w] * a)

        # 3) Kapazität: Slots pro Woche minus belegte Slots durch aktive Therapien
        occupied_slots = len(active)  # 1 Slot pro aktiver Patient*in pro Woche
        free_slots = max(0, cfg.slots_per_week - occupied_slots)

        # 4) Start: so viele wie freie Slots und Queue hergibt
        can_start = min(free_slots, len(queue))
        for _ in range(can_start):
            arrival_week = queue.pop(0)
            wait = w - arrival_week
            wait_times.append(wait)
            active.append(cfg.therapy_duration_weeks)
            total_started += 1

        # 5) Monitoring
        qlen = len(queue)
        trace_queue.append((w, qlen))
        queue_len_sum += qlen

        util = (occupied_slots / cfg.slots_per_week) if cfg.slots_per_week > 0 else 0.0
        trace_util.append((w, util))
        util_sum += util
        util_count += 1

    # Kennzahlen
    if wait_times:
        wt_sorted = sorted(wait_times)
        mean_wait = sum(wait_times) / len(wait_times)
        p90 = wt_sorted[int(0.9 * (len(wt_sorted) - 1))]
        max_wait = wt_sorted[-1]
    else:
        mean_wait, p90, max_wait = 0.0, 0.0, 0

    mean_queue_len = queue_len_sum / total_weeks_sim
    max_queue_len = max(q for _, q in trace_queue) if trace_queue else 0
    mean_util = util_sum / util_count if util_count else 0.0

    return SimResults(
        started=total_started,
        arrived=total_arrived,
        still_waiting_end=len(queue),
        mean_wait_weeks=mean_wait,
        p90_wait_weeks=float(p90),
        max_wait_weeks=int(max_wait),
        mean_queue_len=float(mean_queue_len),
        max_queue_len=int(max_queue_len),
        mean_utilization=float(mean_util),
        trace_queue=trace_queue,
        trace_util=trace_util,
    )

if __name__ == "__main__":
    cfg = SimConfig(
        weeks=52,
        seed=42,
        slots_per_week=SLOTS_PER_WEEK,
        therapy_duration_weeks=THERAPY_DURATION_WEEKS,
        arrivals=ArrivalConfig(
            mode="poisson",      # "poisson" oder "uniform"
            poisson_mean=2000.0, # <-- HIER einstellen (Durchschnitt Arrivals/Woche)
            uniform_low=1500,
            uniform_high=2500
        ),
    )

    res = run_sim(cfg)

    print("=== VT Simulation ===")
    print(f"SLOTS_PER_WEEK: {cfg.slots_per_week:,}")
    print(f"THERAPY_DURATION_WEEKS: {cfg.therapy_duration_weeks}")
    print(f"ARRIVALS_MODE: {cfg.arrivals.mode}")
    if cfg.arrivals.mode == "poisson":
        print(f"POISSON_MEAN: {cfg.arrivals.poisson_mean}")
    else:
        print(f"UNIFORM_RANGE: [{cfg.arrivals.uniform_low}, {cfg.arrivals.uniform_high}]")

    print("\n--- Ergebnis (Jahr + Abbauphase) ---")
    print(f"Arrived (Jahr): {res.arrived:,}")
    print(f"Started: {res.started:,}")
    print(f"Still waiting at end: {res.still_waiting_end:,}")
    print(f"Mean wait (weeks): {res.mean_wait_weeks:.2f}")
    print(f"P90 wait (weeks): {res.p90_wait_weeks:.0f}")
    print(f"Max wait (weeks): {res.max_wait_weeks}")
    print(f"Mean queue length: {res.mean_queue_len:,.0f}")
    print(f"Max queue length: {res.max_queue_len:,}")
    print(f"Mean utilization: {res.mean_utilization*100:.1f}%")

=== VT Simulation ===
SLOTS_PER_WEEK: 116,100
THERAPY_DURATION_WEEKS: 24
ARRIVALS_MODE: poisson
POISSON_MEAN: 2000.0

--- Ergebnis (Jahr + Abbauphase) ---
Arrived (Jahr): 38,872
Started: 38,872
Still waiting at end: 0
Mean wait (weeks): 0.00
P90 wait (weeks): 0
Max wait (weeks): 0
Mean queue length: 0
Max queue length: 0
Mean utilization: 10.1%


In [4]:
# pip install simpy numpy

import simpy
import numpy as np

# -------------------------
# VT Daten (deins)
# -------------------------
TEILZEIT = 5490
VOLLZEIT = 3510

SLOTS_TEILZEIT = 9   # Sitzungen pro Woche
SLOTS_VOLLZEIT = 19  # Sitzungen pro Woche

SLOTS_PRO_WOCHE = TEILZEIT * SLOTS_TEILZEIT + VOLLZEIT * SLOTS_VOLLZEIT  # 116_190

THERAPIE_DAUER_WOCHEN = 24   # 24 Sitzungen, 1/Woche

# -------------------------
# Simulation-Parameter
# -------------------------
WOCHEN = 52
SEED = 42

# "Arrival per week: zufällig"
# -> arrivals ~ Poisson(mean_arrivals_per_week)
MEAN_ARRIVALS_PER_WEEK = 2000  # <-- HIER anpassen

# Ergebnisse sammeln (wie im PDF: wait_times Liste)  [oai_citation:2‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)
wait_times = []

def patient_process(env, name, slots, rng):
    """Entity-Logik wie im PDF: ankommen -> Ressource anfordern -> warten -> Service -> fertig.  [oai_citation:3‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)"""
    t_ankunft = env.now

    # Ressource anfordern (Slot für diese Woche)
    with slots.request() as req:
        yield req  # warten bis Slot frei ist
        t_start = env.now
        wait_times.append(t_start - t_ankunft)

        # Therapie "blockiert" 1 Slot pro Woche für 24 Wochen
        # (vereinfacht als durchgehende Belegung über 24 Wochen)
        yield env.timeout(THERAPIE_DAUER_WOCHEN)

def setup(env, slots, rng):
    """Erzeugt neue Patient*innen (Arrivals). Wie im PDF 'setup' generiert Entities.  [oai_citation:4‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)"""
    i = 0

    # Wir modellieren "pro Woche kommt eine zufällige Anzahl"
    for w in range(WOCHEN):
        # springe zur nächsten Woche
        yield env.timeout(1)

        # zufällige Arrivals in dieser Woche
        arrivals = rng.poisson(MEAN_ARRIVALS_PER_WEEK)

        for _ in range(arrivals):
            i += 1
            env.process(patient_process(env, f"Patient {i}", slots, rng))

if __name__ == "__main__":
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    # Resource = Sitzungs-Slots pro Woche (Kapazität)
    slots = simpy.Resource(env, capacity=SLOTS_PRO_WOCHE)

    env.process(setup(env, slots, rng))

    # laufen lassen: 52 Wochen + genug Zeit, damit gestartete Therapien fertig werden
    env.run(until=WOCHEN + THERAPIE_DAUER_WOCHEN + 1)

    # Auswertung wie im PDF (mean, quantile, max)  [oai_citation:5‡KAP_4_Dynamische_Simulation.pdf](sediment://file_0000000022f8720eb4886c6329c43dba)
    if len(wait_times) == 0:
        print("Keine Patient*innen gestartet.")
    else:
        wt = np.array(wait_times)
        print("=== VT Ergebnis ===")
        print(f"SLOTS_PRO_WOCHE: {SLOTS_PRO_WOCHE:,}")
        print(f"MEAN_ARRIVALS_PER_WEEK: {MEAN_ARRIVALS_PER_WEEK}")
        print(f"Anzahl gestartete Patient*innen: {len(wt):,}")
        print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
        print(f"95%-Quantil Wartezeit: {np.quantile(wt, 0.95):.2f}")
        print(f"Max Wartezeit: {wt.max():.2f}")

=== VT Ergebnis ===
SLOTS_PRO_WOCHE: 116,100
MEAN_ARRIVALS_PER_WEEK: 2000
Anzahl gestartete Patient*innen: 103,839
Ø Wartezeit (Wochen): 0.00
95%-Quantil Wartezeit: 0.00
Max Wartezeit: 0.00


In [5]:
# VT Simulation (alles-in-einem, copy & paste)
# pip install simpy numpy
#
# Modell (einfach, aber verbessert):
# - Kapazität = Sitzungs-Slots pro Woche (Teilzeit + Vollzeit)
# - Arrivals pro Woche = zufällig (Poisson) + optionales Wachstum
# - Therapiestart = randomisiert (Start-Delay)
# - Therapie = 24 einzelne Termine (1/Woche), jeder Termin belegt einen Slot kurz
# - Parallelität entsteht automatisch über viele Slots (Resource capacity)

import simpy
import numpy as np

# -------------------------
# 1) VT Daten
# -------------------------
TEILZEIT_THERAPEUTEN = 5490   # 61%
VOLLZEIT_THERAPEUTEN = 3510   # 39%

SLOTS_PRO_WOCHE_TEILZEIT = 9  # 20 Std -> 9 Sitzungen/Woche
SLOTS_PRO_WOCHE_VOLLZEIT = 19 # 40 Std -> 19 Sitzungen/Woche

SLOTS_PRO_WOCHE = (
    TEILZEIT_THERAPEUTEN * SLOTS_PRO_WOCHE_TEILZEIT
    + VOLLZEIT_THERAPEUTEN * SLOTS_PRO_WOCHE_VOLLZEIT
)  # = 116_190

# Therapie: 24 Sitzungen, 1 Sitzung / Woche
THERAPIE_SITZUNGEN = 24

# Dauer einer einzelnen Sitzung als Wochenanteil (nur damit es "Servicezeit" gibt)
# Du kannst das auch auf 0 setzen, dann ist die Sitzung quasi "instant".
SESSION_DURATION = 1 / 19  # ~0.0526 Woche

# -------------------------
# 2) Simulation Parameter
# -------------------------
WOCHEN_JAHR = 52
SEED = 42

# Arrivals pro Woche: zufällig (Poisson)
BASE_ARRIVALS_MEAN = 2000     # <-- HIER anpassen (Durchschnitt Woche 1)
GROWTH_PER_WEEK = 30          # <-- optional: mittleres Wachstum pro Woche (0 = kein Wachstum)

# Variation / Randomisierung Therapiestart:
# - "uniform": 0..MAX_START_DELAY_WEEKS Wochen
# - "exponential": exp(mean=START_DELAY_MEAN_WEEKS)
START_DELAY_MODE = "uniform"  # "uniform" oder "exponential"
MAX_START_DELAY_WEEKS = 4
START_DELAY_MEAN_WEEKS = 1.5

# Monitoring (für einfache Auswertung)
queue_len_trace = []
util_trace = []
wait_times = []

# -------------------------
# 3) Prozesse
# -------------------------
def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))  # inkl. MAX
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform' oder 'exponential' sein")

def patient(env: simpy.Environment, slots: simpy.Resource, rng: np.random.Generator):
    t_arrival = env.now

    # Randomisierung beim Therapiestart
    delay = sample_start_delay(rng)
    yield env.timeout(delay)

    # Therapie als 24 einzelne Termine/Slots (1 pro Woche)
    for s in range(THERAPIE_SITZUNGEN):
        with slots.request() as req:
            yield req

            # Wartezeit nur bis zur ersten Sitzung messen
            if s == 0:
                wait_times.append(env.now - t_arrival)

            # "Sitzung dauert kurz" (Servicezeit)
            if SESSION_DURATION > 0:
                yield env.timeout(SESSION_DURATION)

        # Rest der Woche bis zum nächsten Termin
        rest = 1.0 - SESSION_DURATION
        if rest > 0:
            yield env.timeout(rest)

def arrivals(env: simpy.Environment, slots: simpy.Resource, rng: np.random.Generator):
    # pro Woche kommt eine zufällige Anzahl Patient*innen
    for w in range(WOCHEN_JAHR):
        yield env.timeout(1.0)

        mean = BASE_ARRIVALS_MEAN + GROWTH_PER_WEEK * w
        n = int(rng.poisson(max(0.0, mean)))

        for _ in range(n):
            env.process(patient(env, slots, rng))

def monitor(env: simpy.Environment, slots: simpy.Resource):
    # misst pro Woche Queue-Länge und Auslastung
    while True:
        q_len = len(slots.queue)
        in_service = slots.count  # aktuell belegte "Slots" (laufende Requests)
        util = (in_service / SLOTS_PRO_WOCHE) if SLOTS_PRO_WOCHE > 0 else 0.0
        queue_len_trace.append((env.now, q_len))
        util_trace.append((env.now, util))
        yield env.timeout(1.0)

# -------------------------
# 4) Run + Output
# -------------------------
def main():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    # Resource = Sitzungs-Slots pro Woche (Kapazität)
    slots = simpy.Resource(env, capacity=SLOTS_PRO_WOCHE)

    env.process(arrivals(env, slots, rng))
    env.process(monitor(env, slots))

    # Jahr + Puffer, damit Therapien fertig laufen
    env.run(until=WOCHEN_JAHR + THERAPIE_SITZUNGEN + 10)

    # Auswertung
    print("=== VT Simulation (alles-in-einem) ===")
    print(f"Slots pro Woche: {SLOTS_PRO_WOCHE:,}")
    print(f"Therapie: {THERAPIE_SITZUNGEN} Sitzungen (1/Woche)")
    print(f"Arrivals: Poisson(mean=BASE + growth*w), BASE={BASE_ARRIVALS_MEAN}, growth={GROWTH_PER_WEEK}/Woche")
    print(f"Start-Delay: {START_DELAY_MODE}")

    if not wait_times:
        print("\nKeine Patient*innen gestartet (oder Wartezeiten gemessen).")
        return

    wt = np.array(wait_times)
    print("\n--- Wartezeiten bis 1. Sitzung ---")
    print(f"Starts (gemessen): {len(wt):,}")
    print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
    print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
    print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
    print(f"Max: {wt.max():.2f}")

    q = np.array([x[1] for x in queue_len_trace]) if queue_len_trace else np.array([])
    u = np.array([x[1] for x in util_trace]) if util_trace else np.array([])
    if len(q) and len(u):
        print("\n--- System-Trace (pro Woche) ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")
        print(f"Ø Auslastung: {u.mean()*100:.1f}%")
        print(f"Max Auslastung: {u.max()*100:.1f}%")

if __name__ == "__main__":
    main()

=== VT Simulation (alles-in-einem) ===
Slots pro Woche: 116,100
Therapie: 24 Sitzungen (1/Woche)
Arrivals: Poisson(mean=BASE + growth*w), BASE=2000, growth=30/Woche
Start-Delay: uniform

--- Wartezeiten bis 1. Sitzung ---
Starts (gemessen): 143,958
Ø Wartezeit (Wochen): 2.00
90% Quantil: 4.00
95% Quantil: 4.00
Max: 4.00

--- System-Trace (pro Woche) ---
Ø Queue-Länge: 0.0
Max Queue-Länge: 0
Ø Auslastung: 1.2%
Max Auslastung: 2.5%


In [6]:
# VT Simulation (realitätsnäher: Therapieplätze / Warteschlange / feste Parallelkapazität)
# Copy & paste.  pip install simpy numpy
#
# Verbesserungen integriert:
# 1) Therapieplätze statt Sitzungs-Slots: Jede Therapie blockiert 1 Platz für 24 Wochen.
# 2) Feste Parallelkapazität: pro Therapeut*in max. parallele Therapien = Sitzungen/Woche (1 Sitzung/Patient/Woche).
# 3) Start nur bei freiem Platz: Warteschlange (FIFO) entscheidet.
# 4) Nachfrage > Kapazität: Beispiel-Parameter so gesetzt, dass es Engpass geben kann (du kannst anpassen).

import simpy
import numpy as np

# -------------------------
# 1) VT Daten
# -------------------------
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510

# Bei 1 Sitzung/Patient/Woche gilt:
# parallele Therapien pro Therapeut*in ≈ Sitzungen pro Woche
PARALLEL_CAP_TEILZEIT = 9    # 20 Std -> 9 Patient*innen parallel
PARALLEL_CAP_VOLLZEIT = 19   # 40 Std -> 19 Patient*innen parallel

# Gesamtzahl Therapieplätze (gleichzeitig behandelbare Patient*innen)
THERAPIEPLAETZE = (
    TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT
    + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
)

# Therapie: 24 Sitzungen, 1/Woche -> 24 Wochen Belegung eines Platzes
THERAPIE_DAUER_WOCHEN = 24

# -------------------------
# 2) Simulation Parameter
# -------------------------
WOCHEN_JAHR = 52
SEED = 42

# Ankünfte pro Woche (zufällig):
# Damit Nachfrage > Kapazität möglich ist:
# grobe Faust: mittlere gestartete Therapien/Woche ~ THERAPIEPLAETZE / THERAPIE_DAUER_WOCHEN
# = 116_190 / 24 ≈ 4_841 Starts/Woche "bei voller Auslastung".
# Setze ARRIVALS_MEAN > ~4_800, dann kann Warteschlange entstehen.
ARRIVALS_BASE_MEAN = 6000     # <-- hier anpassen (Woche 1)
ARRIVALS_GROWTH_PER_WEEK = 50 # <-- optionales Wachstum (0 = konstant)

# Variation beim Therapiestart (administrativer Delay):
START_DELAY_MODE = "uniform"  # "uniform" oder "exponential" oder "none"
MAX_START_DELAY_WEEKS = 2
START_DELAY_MEAN_WEEKS = 1.0

# Monitoring
wait_times = []         # Wartezeit bis Platz frei + Start-Delay
queue_len_trace = []    # Warteschlangenlänge pro Woche
util_trace = []         # Auslastung Therapieplätze pro Woche (0..1)

# -------------------------
# 3) Hilfsfunktionen
# -------------------------
def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))  # 0..MAX
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# -------------------------
# 4) SimPy Prozesse (PDF-Stil: Entity -> request -> wait -> service -> release)
# -------------------------
def patient(env: simpy.Environment, name: str, therapieplaetze: simpy.Resource, rng: np.random.Generator):
    t_arrival = env.now

    # Optional: exogene Verzögerung bis Patient "bereit" ist zu starten (Diagnostik, Antrag, etc.)
    delay = sample_start_delay(rng)
    if delay > 0:
        yield env.timeout(delay)

    # Warteschlange + Start nur wenn Platz frei
    with therapieplaetze.request() as req:
        yield req  # wartet bis ein Therapieplatz frei wird (FIFO)
        t_start = env.now
        wait_times.append(t_start - t_arrival)

        # Therapie blockiert den Platz 24 Wochen am Stück
        yield env.timeout(THERAPIE_DAUER_WOCHEN)

def arrivals(env: simpy.Environment, therapieplaetze: simpy.Resource, rng: np.random.Generator):
    i = 0
    for w in range(WOCHEN_JAHR):
        yield env.timeout(1.0)  # nächste Woche

        mean = ARRIVALS_BASE_MEAN + ARRIVALS_GROWTH_PER_WEEK * w
        n = int(rng.poisson(max(0.0, mean)))

        for _ in range(n):
            i += 1
            env.process(patient(env, f"P{i}", therapieplaetze, rng))

def monitor(env: simpy.Environment, therapieplaetze: simpy.Resource):
    while True:
        q_len = len(therapieplaetze.queue)
        in_service = therapieplaetze.count  # belegte Therapieplätze = laufende Therapien
        util = in_service / THERAPIEPLAETZE if THERAPIEPLAETZE > 0 else 0.0

        queue_len_trace.append((env.now, q_len))
        util_trace.append((env.now, util))
        yield env.timeout(1.0)

# -------------------------
# 5) Run + Auswertung
# -------------------------
def main():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    # Resource = Therapieplätze (gleichzeitig laufende Therapien)
    therapieplaetze = simpy.Resource(env, capacity=THERAPIEPLAETZE)

    env.process(arrivals(env, therapieplaetze, rng))
    env.process(monitor(env, therapieplaetze))

    # Simulation läuft: Jahr + Puffer, damit Therapien aus dem Jahr auch enden können
    env.run(until=WOCHEN_JAHR + THERAPIE_DAUER_WOCHEN + 10)

    print("=== VT Simulation (Therapieplätze, realitätsnäher) ===")
    print(f"Therapieplätze (gleichzeitig): {THERAPIEPLAETZE:,}")
    print(f"Therapiedauer: {THERAPIE_DAUER_WOCHEN} Wochen (blockiert Platz)")
    print(f"Arrivals: Poisson(mean=BASE + growth*w), BASE={ARRIVALS_BASE_MEAN}, growth={ARRIVALS_GROWTH_PER_WEEK}/Woche")
    print(f"Start-Delay: {START_DELAY_MODE}")

    if not wait_times:
        print("\nKeine Starts gemessen.")
        return

    wt = np.array(wait_times)
    print("\n--- Wartezeit bis Therapiebeginn (inkl. Start-Delay) ---")
    print(f"Starts (gemessen): {len(wt):,}")
    print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
    print(f"Median: {np.quantile(wt, 0.50):.2f}")
    print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
    print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
    print(f"Max: {wt.max():.2f}")

    q = np.array([x[1] for x in queue_len_trace]) if queue_len_trace else np.array([])
    u = np.array([x[1] for x in util_trace]) if util_trace else np.array([])
    if len(q) and len(u):
        print("\n--- System-Trace (pro Woche) ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")
        print(f"Ø Auslastung Plätze: {u.mean()*100:.1f}%")
        print(f"Max Auslastung Plätze: {u.max()*100:.1f}%")

    # Zusatz: theoretische Start-Kapazität bei voller Auslastung
    start_capacity_per_week = THERAPIEPLAETZE / THERAPIE_DAUER_WOCHEN
    print("\n--- Faustformel ---")
    print(f"Max. Starts/Woche (≈ Plätze / Dauer): {start_capacity_per_week:,.0f}")
    print("Wenn ARRIVALS_MEAN deutlich darüber liegt, baut sich eine Warteschlange auf.")

if __name__ == "__main__":
    main()

=== VT Simulation (Therapieplätze, realitätsnäher) ===
Therapieplätze (gleichzeitig): 116,100
Therapiedauer: 24 Wochen (blockiert Platz)
Arrivals: Poisson(mean=BASE + growth*w), BASE=6000, growth=50/Woche
Start-Delay: uniform

--- Wartezeit bis Therapiebeginn (inkl. Start-Delay) ---
Starts (gemessen): 378,111
Ø Wartezeit (Wochen): 9.91
Median: 8.00
90% Quantil: 19.00
95% Quantil: 25.00
Max: 26.00

--- System-Trace (pro Woche) ---
Ø Queue-Länge: 40568.3
Max Queue-Länge: 119106
Ø Auslastung Plätze: 84.5%
Max Auslastung Plätze: 100.0%

--- Faustformel ---
Max. Starts/Woche (≈ Plätze / Dauer): 4,838
Wenn ARRIVALS_MEAN deutlich darüber liegt, baut sich eine Warteschlange auf.


In [7]:
# VT Simulation (kombiniert: Kapazität wie Option 1 + Ankünfte/Gate wie Option 2 + lineares Wachstum + Start-Delay)
# Copy & paste | pip install simpy numpy
#
# Kombiniert:
# 1) Kapazität = Therapieplätze (parallele Therapien) aus Teilzeit/Vollzeit: 5.490*9 + 3.510*19 = 116.100
# 2) Kontinuierliche Poisson-Ankünfte über exponential Interarrival (Patient*innen kommen auch "zwischen Wochen")
# 3) Nachfrage-Dynamik linear pro Woche: rate(t) = BASE + GROWTH_PER_WEEK * t
# 4) Gate/Filter: nur mit Wahrscheinlichkeit P_START_THERAPY bleibt Patient im System
# 5) Wartezeit bis Therapiebeginn (inkl. optionalem Start-Delay)
#
# Zeitbasis: Wochen

import simpy
import numpy as np

# -------------------------
# 1) Kapazität (Therapieplätze = parallele Therapien)
# -------------------------
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510
PARALLEL_CAP_TEILZEIT = 9
PARALLEL_CAP_VOLLZEIT = 19

THERAPIEPLAETZE = TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
# = 116_100

THERAPIE_DAUER_WOCHEN = 24  # blockiert 1 Platz durchgehend

# -------------------------
# 2) Nachfrage + Gate (Ja/Nein)
# -------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = WEEKS_PER_YEAR + THERAPIE_DAUER_WOCHEN + 10
SEED = 42

TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR  # ~0.888

# -------------------------
# 3) Ankunftsrate: kontinuierlich + linearer Trend pro Woche
# -------------------------
# BASE: mittlere Ankünfte pro Woche zu Beginn
ARRIVALS_BASE_PER_WEEK = 6000.0   # <-- anpassen
# linearer Zuwachs pro Woche
ARRIVALS_GROWTH_PER_WEEK = 50.0   # <-- anpassen (0 = konstant)

def arrival_rate_per_week(t_week: float) -> float:
    """Linear wachsende Rate (pro Woche)."""
    return max(0.0, ARRIVALS_BASE_PER_WEEK + ARRIVALS_GROWTH_PER_WEEK * t_week)

# -------------------------
# 4) Optional: Start-Delay (administrativ)
# -------------------------
START_DELAY_MODE = "uniform"  # "uniform" | "exponential" | "none"
MAX_START_DELAY_WEEKS = 2
START_DELAY_MEAN_WEEKS = 1.0

def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# -------------------------
# 5) Ergebnis-Speicher
# -------------------------
wait_times = []       # Wartezeit bis Therapiebeginn (inkl. Start-Delay)
queue_len_trace = []  # Warteschlangenlänge über Zeit
util_trace = []       # Auslastung Therapieplätze über Zeit (0..1)
dropped = 0           # Gate: rausgefiltert
arrived_total = 0     # alle Ankünfte (vor Gate)
accepted_total = 0    # nach Gate

# -------------------------
# 6) SimPy Prozesse
# -------------------------
def patient(env: simpy.Environment, therapy_slots: simpy.Resource, rng: np.random.Generator):
    global dropped, accepted_total

    t_arrival = env.now

    # Gate/Filter (Ja/Nein)
    if rng.random() > P_START_THERAPY:
        dropped += 1
        return
    accepted_total += 1

    # optionaler Start-Delay (z.B. Antrag/Diagnostik)
    delay = sample_start_delay(rng)
    if delay > 0:
        yield env.timeout(delay)

    # Warteschlange + Start nur bei freiem Therapieplatz
    with therapy_slots.request() as req:
        yield req
        t_start = env.now
        wait_times.append(t_start - t_arrival)

        # Therapie blockiert den Platz 24 Wochen
        yield env.timeout(THERAPIE_DAUER_WOCHEN)

def arrivals_continuous(env: simpy.Environment, therapy_slots: simpy.Resource, rng: np.random.Generator):
    """
    Kontinuierliche Ankünfte:
    - nächstes Interarrival ~ Exp(rate(t))
    - rate(t) in 1/Woche (linear über t)
    """
    global arrived_total
    i = 0

    while env.now < WEEKS_PER_YEAR:
        lam = arrival_rate_per_week(env.now)  # pro Woche
        if lam <= 0:
            # falls Rate 0 ist, springe 1 Woche weiter
            yield env.timeout(1.0)
            continue

        interarrival = rng.exponential(scale=1.0 / lam)
        yield env.timeout(interarrival)

        if env.now >= WEEKS_PER_YEAR:
            break

        arrived_total += 1
        i += 1
        env.process(patient(env, therapy_slots, rng))

def monitor(env: simpy.Environment, therapy_slots: simpy.Resource, step: float = 1.0):
    """Monitoring pro Woche (step=1.0)."""
    while True:
        q_len = len(therapy_slots.queue)
        in_service = therapy_slots.count
        util = in_service / THERAPIEPLAETZE if THERAPIEPLAETZE > 0 else 0.0
        queue_len_trace.append((env.now, q_len))
        util_trace.append((env.now, util))
        yield env.timeout(step)

# -------------------------
# 7) Run + Auswertung
# -------------------------
def main():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    therapy_slots = simpy.Resource(env, capacity=THERAPIEPLAETZE)

    env.process(arrivals_continuous(env, therapy_slots, rng))
    env.process(monitor(env, therapy_slots, step=1.0))

    env.run(until=SIMULATION_DURATION)

    print("=== VT Simulation (kombiniert) ===")
    print(f"Therapieplätze (gleichzeitig): {THERAPIEPLAETZE:,}")
    print(f"Therapiedauer: {THERAPIE_DAUER_WOCHEN} Wochen (blockiert Platz)")
    print(f"Arrivals (linear): rate(t)=BASE+GROWTH*t, BASE={ARRIVALS_BASE_PER_WEEK}, GROWTH={ARRIVALS_GROWTH_PER_WEEK}/Woche")
    print(f"Gate P_START_THERAPY: {P_START_THERAPY:.4f}")
    print(f"Start-Delay: {START_DELAY_MODE}")

    print("\n--- Nachfrage / Gate ---")
    print(f"Arrivals (gesamt, vor Gate): {arrived_total:,}")
    print(f"Accepted (nach Gate):        {accepted_total:,}")
    print(f"Dropped (Gate):              {dropped:,} ({(dropped/arrived_total*100 if arrived_total else 0):.1f}%)")

    if not wait_times:
        print("\nKeine Starts gemessen.")
        return

    wt = np.array(wait_times)
    print("\n--- Wartezeit bis Therapiebeginn (inkl. Start-Delay) ---")
    print(f"Starts (gemessen): {len(wt):,}")
    print(f"Ø Wartezeit (Wochen): {wt.mean():.2f}")
    print(f"Median: {np.quantile(wt, 0.50):.2f}")
    print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
    print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
    print(f"Max: {wt.max():.2f}")

    q = np.array([x[1] for x in queue_len_trace]) if queue_len_trace else np.array([])
    u = np.array([x[1] for x in util_trace]) if util_trace else np.array([])
    if len(q) and len(u):
        print("\n--- System-Trace (pro Woche) ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")
        print(f"Ø Auslastung Plätze: {u.mean()*100:.1f}%")
        print(f"Max Auslastung Plätze: {u.max()*100:.1f}%")

    # Faustformel: Max. Starts/Woche bei voller Auslastung
    start_capacity_per_week = THERAPIEPLAETZE / THERAPIE_DAUER_WOCHEN
    print("\n--- Faustformel ---")
    print(f"Max. Starts/Woche (≈ Plätze / Dauer): {start_capacity_per_week:,.0f}")
    print("Wenn (Accepted Arrivals/Woche) langfristig darüber liegt, wächst die Warteschlange.")

if __name__ == "__main__":
    main()

=== VT Simulation (kombiniert) ===
Therapieplätze (gleichzeitig): 116,100
Therapiedauer: 24 Wochen (blockiert Platz)
Arrivals (linear): rate(t)=BASE+GROWTH*t, BASE=6000.0, GROWTH=50.0/Woche
Gate P_START_THERAPY: 0.8882
Start-Delay: uniform

--- Nachfrage / Gate ---
Arrivals (gesamt, vor Gate): 380,162
Accepted (nach Gate):        337,739
Dropped (Gate):              42,423 (11.2%)

--- Wartezeit bis Therapiebeginn (inkl. Start-Delay) ---
Starts (gemessen): 337,739
Ø Wartezeit (Wochen): 6.73
Median: 6.22
90% Quantil: 14.28
95% Quantil: 15.04
Max: 16.10

--- System-Trace (pro Woche) ---
Ø Queue-Länge: 22534.4
Max Queue-Länge: 81709
Ø Auslastung Plätze: 80.1%
Max Auslastung Plätze: 100.0%

--- Faustformel ---
Max. Starts/Woche (≈ Plätze / Dauer): 4,838
Wenn (Accepted Arrivals/Woche) langfristig darüber liegt, wächst die Warteschlange.


In [8]:
# VT Simulation mit Effizienz-Faktor (Szenarien spielbar)
# pip install simpy numpy
#
# Idee:
# - Theoretische Therapieplätze = (Teilzeit*9 + Vollzeit*19) parallele Therapien
# - Effektive Therapieplätze = theoretische Therapieplätze * EFFICIENCY_FACTOR
# - EFFICIENCY_FACTOR kann aus Arbeitswochen kommen (z.B. 42/52) UND/oder zusätzlich reduziert werden (Admin, No-Shows etc.)
#
# Kombiniert:
# - Kapazität wie Option 1 (Therapieplätze aus Teilzeit/Vollzeit)
# - Ankünfte kontinuierlich (exponential Interarrival, Poisson-Prozess)
# - lineares Wachstum der Rate (BASE + growth*t)
# - Gate/Filter (P_START_THERAPY)
# - Wartezeit bis Therapiebeginn (optional inkl. Start-Delay)

import simpy
import numpy as np

# -------------------------
# 1) VT Kapazität (theoretisch)
# -------------------------
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510
PARALLEL_CAP_TEILZEIT = 9
PARALLEL_CAP_VOLLZEIT = 19

THEORETISCHE_THERAPIEPLAETZE = (
    TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT
    + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
)  # 116_100

THERAPIE_DAUER_WOCHEN = 24

# -------------------------
# 2) Effizienz-Faktor (hier spielst du mit Szenarien)
# -------------------------
# a) Arbeitswochen-Logik (wie in deinem Board z.B. 52 - 10 = 42)
WEEKS_PER_YEAR = 52
OFF_WEEKS = 10                       # Urlaub + Krankheit/Fortbildung + Feiertage (Beispiel)
WORK_WEEKS = WEEKS_PER_YEAR - OFF_WEEKS
FACTOR_WORKWEEKS = WORK_WEEKS / WEEKS_PER_YEAR  # z.B. 42/52 ≈ 0.8077

# b) Zusätzlicher Effizienz-Abschlag (Admin, Doku, Ausfälle, Leerlauf, No-shows, etc.)
# 1.0 = kein Abschlag, 0.9 = 10% weniger effektiv, 0.8 = 20% weniger effektiv
FACTOR_OTHER = 0.90

# c) Gesamt-Effizienzfaktor
EFFICIENCY_FACTOR = FACTOR_WORKWEEKS * FACTOR_OTHER
# Du kannst auch direkt setzen, z.B.: EFFICIENCY_FACTOR = 0.70

def effective_capacity(theoretical_places: int, eff: float) -> int:
    return max(1, int(round(theoretical_places * eff)))

# -------------------------
# 3) Nachfrage + Gate
# -------------------------
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR  # ~0.8882

# -------------------------
# 4) Ankünfte: kontinuierlich + lineares Wachstum
# -------------------------
ARRIVALS_BASE_PER_WEEK = 6000.0     # mittlere Kontakte/Woche zu Beginn
ARRIVALS_GROWTH_PER_WEEK = 50.0     # Zuwachs pro Woche

def arrival_rate_per_week(t_week: float) -> float:
    return max(0.0, ARRIVALS_BASE_PER_WEEK + ARRIVALS_GROWTH_PER_WEEK * t_week)

# -------------------------
# 5) Optional: Start-Delay (administrativ)
# -------------------------
START_DELAY_MODE = "uniform"  # "uniform" | "exponential" | "none"
MAX_START_DELAY_WEEKS = 2
START_DELAY_MEAN_WEEKS = 1.0

def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# -------------------------
# 6) Simulation Settings
# -------------------------
SEED = 42
SIMULATION_DURATION = WEEKS_PER_YEAR + THERAPIE_DAUER_WOCHEN + 10  # Jahr + Puffer

# -------------------------
# 7) SimPy Modell (Funktionen)
# -------------------------
def run_scenario(
    scenario_name: str,
    efficiency_factor: float,
    seed: int = SEED,
):
    rng = np.random.default_rng(seed)
    env = simpy.Environment()

    capacity = effective_capacity(THEORETISCHE_THERAPIEPLAETZE, efficiency_factor)
    therapy_slots = simpy.Resource(env, capacity=capacity)

    # Stats
    wait_times = []
    queue_len_trace = []
    util_trace = []
    arrived_total = 0
    accepted_total = 0
    dropped_total = 0

    def patient():
        nonlocal accepted_total, dropped_total
        t_arrival = env.now

        # Gate
        if rng.random() > P_START_THERAPY:
            dropped_total += 1
            return
        accepted_total += 1

        # optionaler Delay
        d = sample_start_delay(rng)
        if d > 0:
            yield env.timeout(d)

        # Warteschlange -> Start nur bei freiem Platz
        with therapy_slots.request() as req:
            yield req
            wait_times.append(env.now - t_arrival)
            yield env.timeout(THERAPIE_DAUER_WOCHEN)

    def arrivals_continuous():
        nonlocal arrived_total
        while env.now < WEEKS_PER_YEAR:
            lam = arrival_rate_per_week(env.now)
            if lam <= 0:
                yield env.timeout(1.0)
                continue

            interarrival = rng.exponential(scale=1.0 / lam)
            yield env.timeout(interarrival)

            if env.now >= WEEKS_PER_YEAR:
                break

            arrived_total += 1
            env.process(patient())

    def monitor(step: float = 1.0):
        while True:
            q_len = len(therapy_slots.queue)
            in_service = therapy_slots.count
            util = in_service / capacity if capacity > 0 else 0.0
            queue_len_trace.append((env.now, q_len))
            util_trace.append((env.now, util))
            yield env.timeout(step)

    env.process(arrivals_continuous())
    env.process(monitor(step=1.0))
    env.run(until=SIMULATION_DURATION)

    # Auswertung
    wt = np.array(wait_times) if wait_times else np.array([])
    q = np.array([x[1] for x in queue_len_trace]) if queue_len_trace else np.array([])
    u = np.array([x[1] for x in util_trace]) if util_trace else np.array([])

    print(f"\n=== Szenario: {scenario_name} ===")
    print(f"Theoretische Plätze: {THEORETISCHE_THERAPIEPLAETZE:,}")
    print(f"Effizienz-Faktor: {efficiency_factor:.3f}")
    print(f"Effektive Plätze: {capacity:,}")
    print(f"Arbeitswochen-Faktor: {FACTOR_WORKWEEKS:.3f} (Work weeks={WORK_WEEKS}/{WEEKS_PER_YEAR})")
    print(f"Other-Faktor: {FACTOR_OTHER:.3f}")
    print(f"Therapiedauer: {THERAPIE_DAUER_WOCHEN} Wochen")
    print(f"Arrivals-Rate: BASE={ARRIVALS_BASE_PER_WEEK}, GROWTH={ARRIVALS_GROWTH_PER_WEEK}/Woche")
    print(f"Gate P_START_THERAPY: {P_START_THERAPY:.4f}")
    print(f"Start-Delay: {START_DELAY_MODE}")

    print("\n--- Nachfrage / Gate ---")
    print(f"Arrivals (vor Gate): {arrived_total:,}")
    print(f"Accepted (nach Gate): {accepted_total:,}")
    print(f"Dropped: {dropped_total:,} ({(dropped_total/arrived_total*100 if arrived_total else 0):.1f}%)")

    if wt.size == 0:
        print("\nKeine Starts gemessen.")
        return

    print("\n--- Wartezeit bis Therapiebeginn (inkl. Delay) ---")
    print(f"Starts: {wt.size:,}")
    print(f"Ø Wartezeit: {wt.mean():.2f} Wochen")
    print(f"Median: {np.quantile(wt, 0.50):.2f}")
    print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
    print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
    print(f"Max: {wt.max():.2f}")

    if q.size and u.size:
        print("\n--- System ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")
        print(f"Ø Auslastung: {u.mean()*100:.1f}%")
        print(f"Max Auslastung: {u.max()*100:.1f}%")

    # Faustformel mit effektiver Kapazität
    max_starts_per_week = capacity / THERAPIE_DAUER_WOCHEN
    print("\n--- Faustformel (mit Effizienz) ---")
    print(f"Max Starts/Woche ≈ {max_starts_per_week:,.0f}")
    print("Wenn (Accepted Arrivals/Woche) langfristig darüber liegt -> Warteschlange wächst.")

# -------------------------
# 8) Hauptteil: Szenarien definieren
# -------------------------
if __name__ == "__main__":
    print("Starte VT-Simulation mit Effizienz-Faktor...")

    # Du kannst hier beliebige Szenarien anlegen:
    scenarios = [
        ("Optimistisch (eff=0.90)", 0.90),
        ("Baseline (Workweeks*Other)", EFFICIENCY_FACTOR),
        ("Pessimistisch (eff=0.60)", 0.60),
    ]

    for name, eff in scenarios:
        run_scenario(name, eff, seed=SEED)

Starte VT-Simulation mit Effizienz-Faktor...

=== Szenario: Optimistisch (eff=0.90) ===
Theoretische Plätze: 116,100
Effizienz-Faktor: 0.900
Effektive Plätze: 104,490
Arbeitswochen-Faktor: 0.808 (Work weeks=42/52)
Other-Faktor: 0.900
Therapiedauer: 24 Wochen
Arrivals-Rate: BASE=6000.0, GROWTH=50.0/Woche
Gate P_START_THERAPY: 0.8882
Start-Delay: uniform

--- Nachfrage / Gate ---
Arrivals (vor Gate): 380,162
Accepted (nach Gate): 337,739
Dropped: 42,423 (11.2%)

--- Wartezeit bis Therapiebeginn (inkl. Delay) ---
Starts: 337,739
Ø Wartezeit: 9.72 Wochen
Median: 8.35
90% Quantil: 18.50
95% Quantil: 24.42
Max: 26.09

--- System ---
Ø Queue-Länge: 34277.9
Max Queue-Länge: 104929
Ø Auslastung: 85.4%
Max Auslastung: 100.0%

--- Faustformel (mit Effizienz) ---
Max Starts/Woche ≈ 4,354
Wenn (Accepted Arrivals/Woche) langfristig darüber liegt -> Warteschlange wächst.

=== Szenario: Baseline (Workweeks*Other) ===
Theoretische Plätze: 116,100
Effizienz-Faktor: 0.727
Effektive Plätze: 84,396
Arbeits

In [9]:
# VT Simulation (einfach, aber realitätsnäher) + Effizienz-Faktor + exponentielles Wachstum + Kipppunkt
# pip install simpy numpy
#
# Modell:
# - Kapazität = Therapieplätze (parallele Therapien) aus Teilzeit/Vollzeit
# - Effizienz-Faktor reduziert theoretische Plätze (Urlaub/Krankheit/Admin etc.)
# - Ankünfte: kontinuierlich (Poisson-Prozess) über exponential Interarrival
# - Nachfrage wächst exponentiell: lambda(t) = lambda0 * exp(g*t)
# - Gate: nur Anteil (Ja) bleibt im System
# - Wartezeit: Anfrage -> Therapiebeginn (inkl. optionalem Start-Delay)
# - Kipppunkt (Woche) wird automatisch bestimmt

import simpy
import numpy as np

# -------------------------
# 1) Kapazität: theoretische Therapieplätze
# -------------------------
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510
PARALLEL_CAP_TEILZEIT = 9
PARALLEL_CAP_VOLLZEIT = 19

THEORETISCHE_PLAETZE = (
    TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT
    + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
)  # 116_100

THERAPIE_DAUER_WOCHEN = 24

# -------------------------
# 2) Effizienz-Faktor (zum Spielen für Szenarien)
# -------------------------
WEEKS_PER_YEAR = 52
OFF_WEEKS = 10                 # z.B. Urlaub + Krankheit/Fortbildung + Feiertage
FACTOR_WORKWEEKS = (WEEKS_PER_YEAR - OFF_WEEKS) / WEEKS_PER_YEAR  # z.B. 42/52 ≈ 0.808

FACTOR_OTHER = 0.90            # z.B. Doku/Admin/No-shows (1.0 = kein Abschlag)

EFFICIENCY_FACTOR = FACTOR_WORKWEEKS * FACTOR_OTHER
# Du kannst auch direkt setzen: EFFICIENCY_FACTOR = 0.70

def effective_capacity(theoretical_places: int, eff: float) -> int:
    return max(1, int(round(theoretical_places * eff)))

# -------------------------
# 3) Nachfrage + Gate
# -------------------------
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR  # ~0.888

# -------------------------
# 4) Ankünfte: exponentielles Wachstum + kontinuierlicher Poisson-Prozess
# -------------------------
ARRIVALS_BASE_PER_WEEK = 6000.0     # lambda0 (Woche 0)
ARRIVAL_GROWTH_PER_WEEK = 0.008     # g (pro Woche). 0.008 ~ +52% pro Jahr (ungefähr)

def arrival_rate_per_week(t_week: float) -> float:
    # lambda(t) = lambda0 * exp(g*t)
    return ARRIVALS_BASE_PER_WEEK * np.exp(ARRIVAL_GROWTH_PER_WEEK * t_week)

# -------------------------
# 5) Optionaler Start-Delay
# -------------------------
START_DELAY_MODE = "uniform"   # "uniform" | "exponential" | "none"
MAX_START_DELAY_WEEKS = 2
START_DELAY_MEAN_WEEKS = 1.0

def sample_start_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_START_DELAY_WEEKS + 1))
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(START_DELAY_MEAN_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# -------------------------
# 6) Kipppunkt-Definition (einfach & robust)
# -------------------------
# Kipppunkt = erste Woche, ab der die Warteschlange "dauerhaft" > 0 ist
# (z.B. mindestens 3 Wochen am Stück)
KIPPPUNKT_CONSEC_WEEKS = 3

def find_tipping_week(queue_weekly: np.ndarray, consec: int = 3) -> int | None:
    """
    queue_weekly: Array der Queue-Längen pro Woche (Index ~ Woche).
    Gibt die erste Woche zurück, ab der >= consec Wochen in Folge Queue>0 sind.
    """
    run = 0
    for w, q in enumerate(queue_weekly):
        if q > 0:
            run += 1
            if run >= consec:
                return w - consec + 1
        else:
            run = 0
    return None

# -------------------------
# 7) Simulation
# -------------------------
SEED = 42
SIMULATION_DURATION = WEEKS_PER_YEAR + THERAPIE_DAUER_WOCHEN + 10

def main():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    capacity = effective_capacity(THEORETISCHE_PLAETZE, EFFICIENCY_FACTOR)
    therapy_slots = simpy.Resource(env, capacity=capacity)

    # Stats
    wait_times = []
    queue_trace = []    # (time, queue_len)
    util_trace = []     # (time, util)
    arrived_total = 0
    accepted_total = 0
    dropped_total = 0

    def patient():
        nonlocal accepted_total, dropped_total
        t_arrival = env.now

        # Gate
        if rng.random() > P_START_THERAPY:
            dropped_total += 1
            return
        accepted_total += 1

        # optionaler Delay
        d = sample_start_delay(rng)
        if d > 0:
            yield env.timeout(d)

        # Warteschlange / Start nur bei freiem Platz
        with therapy_slots.request() as req:
            yield req
            wait_times.append(env.now - t_arrival)
            yield env.timeout(THERAPIE_DAUER_WOCHEN)

    def arrivals_continuous():
        nonlocal arrived_total
        while env.now < WEEKS_PER_YEAR:
            lam = arrival_rate_per_week(env.now)
            interarrival = rng.exponential(scale=1.0 / lam)
            yield env.timeout(interarrival)

            if env.now >= WEEKS_PER_YEAR:
                break

            arrived_total += 1
            env.process(patient())

    def monitor(step: float = 1.0):
        while True:
            q_len = len(therapy_slots.queue)
            in_service = therapy_slots.count
            util = in_service / capacity
            queue_trace.append((env.now, q_len))
            util_trace.append((env.now, util))
            yield env.timeout(step)

    env.process(arrivals_continuous())
    env.process(monitor(step=1.0))
    env.run(until=SIMULATION_DURATION)

    # -------------------------
    # Auswertung (inkl. Kipppunkt)
    # -------------------------
    wt = np.array(wait_times) if wait_times else np.array([])

    # wöchentliche Arrays (wir loggen pro Woche; env.now ist float, aber praktisch 0..)
    queue_weekly = np.array([q for _, q in queue_trace]) if queue_trace else np.array([])
    util_weekly = np.array([u for _, u in util_trace]) if util_trace else np.array([])

    tipping = find_tipping_week(queue_weekly[:WEEKS_PER_YEAR], consec=KIPPPUNKT_CONSEC_WEEKS)

    print("=== VT Simulation (Effizienz + exponentielle Nachfrage + Kipppunkt) ===")
    print(f"Theoretische Plätze: {THEORETISCHE_PLAETZE:,}")
    print(f"Effizienz-Faktor: {EFFICIENCY_FACTOR:.3f}  (Workweeks={FACTOR_WORKWEEKS:.3f}, Other={FACTOR_OTHER:.3f})")
    print(f"Effektive Plätze: {capacity:,}")
    print(f"Therapiedauer: {THERAPIE_DAUER_WOCHEN} Wochen (blockiert Platz)")
    print(f"Ankunftsrate: lambda(t)=BASE*exp(g*t), BASE={ARRIVALS_BASE_PER_WEEK}, g={ARRIVAL_GROWTH_PER_WEEK}/Woche")
    print(f"Gate P_START_THERAPY: {P_START_THERAPY:.4f}")
    print(f"Start-Delay: {START_DELAY_MODE}")

    print("\n--- Nachfrage / Gate ---")
    print(f"Arrivals (vor Gate): {arrived_total:,}")
    print(f"Accepted (nach Gate): {accepted_total:,}")
    print(f"Dropped: {dropped_total:,} ({(dropped_total/arrived_total*100 if arrived_total else 0):.1f}%)")

    if wt.size > 0:
        print("\n--- Wartezeit bis Therapiebeginn (inkl. Delay) ---")
        print(f"Starts: {wt.size:,}")
        print(f"Ø Wartezeit: {wt.mean():.2f} Wochen")
        print(f"Median: {np.quantile(wt, 0.50):.2f}")
        print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
        print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
        print(f"Max: {wt.max():.2f}")

    if queue_weekly.size and util_weekly.size:
        print("\n--- System (pro Woche, Monitoring) ---")
        print(f"Ø Queue-Länge: {queue_weekly.mean():.1f}")
        print(f"Max Queue-Länge: {queue_weekly.max():.0f}")
        print(f"Ø Auslastung: {util_weekly.mean()*100:.1f}%")
        print(f"Max Auslastung: {util_weekly.max()*100:.1f}%")

    print("\n--- Kipppunkt ---")
    if tipping is None:
        print(f"Kein Kipppunkt gefunden (Queue nicht {KIPPPUNKT_CONSEC_WEEKS} Wochen am Stück > 0 im Jahr).")
    else:
        print(f"Kipppunkt ab Woche {tipping} (Queue > 0 für mindestens {KIPPPUNKT_CONSEC_WEEKS} Wochen in Folge).")

    # Faustformel mit effektiver Kapazität
    max_starts_per_week = capacity / THERAPIE_DAUER_WOCHEN
    print("\n--- Faustformel (mit Effizienz) ---")
    print(f"Max Starts/Woche ≈ {max_starts_per_week:,.0f}")
    print("Wenn (Accepted Arrivals/Woche) langfristig darüber liegt, wächst die Warteschlange.")

if __name__ == "__main__":
    main()

=== VT Simulation (Effizienz + exponentielle Nachfrage + Kipppunkt) ===
Theoretische Plätze: 116,100
Effizienz-Faktor: 0.727  (Workweeks=0.808, Other=0.900)
Effektive Plätze: 84,396
Therapiedauer: 24 Wochen (blockiert Platz)
Ankunftsrate: lambda(t)=BASE*exp(g*t), BASE=6000.0, g=0.008/Woche
Gate P_START_THERAPY: 0.8882
Start-Delay: uniform

--- Nachfrage / Gate ---
Arrivals (vor Gate): 387,633
Accepted (nach Gate): 344,359
Dropped: 43,274 (11.2%)

--- Wartezeit bis Therapiebeginn (inkl. Delay) ---
Starts: 326,421
Ø Wartezeit: 16.62 Wochen
Median: 12.45
90% Quantil: 34.80
95% Quantil: 35.68
Max: 37.23

--- System (pro Woche, Monitoring) ---
Ø Queue-Länge: 66328.7
Max Queue-Länge: 151306
Ø Auslastung: 89.4%
Max Auslastung: 100.0%

--- Kipppunkt ---
Kipppunkt ab Woche 16 (Queue > 0 für mindestens 3 Wochen in Folge).

--- Faustformel (mit Effizienz) ---
Max Starts/Woche ≈ 3,516
Wenn (Accepted Arrivals/Woche) langfristig darüber liegt, wächst die Warteschlange.


In [10]:
# VT Simulation (vereinfacht): Effizienz-Faktor + exponentielles Wachstum + Gate + Kipppunkt
# pip install simpy numpy
#
# Zeitbasis: Wochen
# - Kapazität = Therapieplätze (parallele Therapien) * Effizienz
# - Ankünfte: kontinuierlich (Poisson) mit exponentiellem Wachstum
# - Gate: nur Anteil "Ja" bleibt
# - Therapie blockiert Platz 24 Wochen
# - Kipppunkt: erste Woche, ab der Queue >= 1 für X Wochen am Stück

import simpy
import numpy as np

# ------------------ Inputs ------------------
WEEKS = 52
SEED = 42

# Therapie
THERAPY_WEEKS = 24

# Kapazität (theoretische Therapieplätze aus Teilzeit/Vollzeit)
THEO_PLACES = 5490 * 9 + 3510 * 19  # 116_100

# Effizienz-Faktor (hiermit "spielen")
OFF_WEEKS = 10          # z.B. Urlaub/Krankheit/Fortbildung/Feiertage
FACTOR_OTHER = 0.90     # z.B. Admin/No-shows
EFF = ((WEEKS - OFF_WEEKS) / WEEKS) * FACTOR_OTHER
PLACES = max(1, int(round(THEO_PLACES * EFF)))

# Nachfrage
P_START = 84557 / 95201          # Gate ~0.888
LAMBDA0 = 6000.0                 # Ankünfte/Woche am Anfang
GROWTH = 0.008                   # exponentielles Wachstum pro Woche: lambda(t)=LAMBDA0*exp(GROWTH*t)

# Optionaler Start-Delay (0..2 Wochen). Setz MAX_DELAY=0 für aus.
MAX_DELAY = 2

# Kipppunkt-Regel
CONSEC_WEEKS_FOR_TIP = 3

# ------------------ Simulation ------------------
def lambda_t(t):  # Rate pro Woche
    return LAMBDA0 * np.exp(GROWTH * t)

def run():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    slots = simpy.Resource(env, capacity=PLACES)

    wait_times = []
    queue_weekly = []  # Queue-Länge pro Woche (0..WEEKS-1)
    arrived = accepted = dropped = 0

    def patient():
        nonlocal accepted, dropped
        t_arr = env.now

        # Gate
        if rng.random() > P_START:
            dropped += 1
            return
        accepted += 1

        # optionaler Delay
        if MAX_DELAY > 0:
            yield env.timeout(float(rng.integers(0, MAX_DELAY + 1)))

        # warten bis Platz frei, dann Therapie starten
        with slots.request() as req:
            yield req
            wait_times.append(env.now - t_arr)
            yield env.timeout(THERAPY_WEEKS)

    def arrivals():
        nonlocal arrived
        while env.now < WEEKS:
            inter = rng.exponential(1.0 / lambda_t(env.now))
            yield env.timeout(inter)
            if env.now >= WEEKS:
                break
            arrived += 1
            env.process(patient())

    def monitor():
        # einmal pro Woche Queue-Länge speichern
        for _ in range(WEEKS):
            queue_weekly.append(len(slots.queue))
            yield env.timeout(1.0)

    env.process(arrivals())
    env.process(monitor())

    # Jahr + Puffer, damit Therapien auslaufen können
    env.run(until=WEEKS + THERAPY_WEEKS + 10)

    # Kipppunkt: erste Woche mit Queue>0 für CONSEC_WEEKS_FOR_TIP Wochen am Stück
    tip = None
    runlen = 0
    for w, q in enumerate(queue_weekly):
        runlen = runlen + 1 if q > 0 else 0
        if runlen >= CONSEC_WEEKS_FOR_TIP:
            tip = w - CONSEC_WEEKS_FOR_TIP + 1
            break

    # Output
    wt = np.array(wait_times) if wait_times else np.array([])
    print("=== VT (vereinfacht) ===")
    print(f"Theo Plätze: {THEO_PLACES:,} | Eff: {EFF:.3f} -> Plätze effektiv: {PLACES:,}")
    print(f"Therapie: {THERAPY_WEEKS} Wochen | Gate P_START: {P_START:.3f}")
    print(f"Nachfrage: lambda0={LAMBDA0}, growth={GROWTH}/Woche (exponentiell)")
    print(f"Arrivals: {arrived:,} | Accepted: {accepted:,} | Dropped: {dropped:,}")

    if wt.size:
        print(f"Ø Wartezeit: {wt.mean():.2f} Wochen | Median: {np.quantile(wt,0.5):.2f} | 95%: {np.quantile(wt,0.95):.2f} | Max: {wt.max():.2f}")
    else:
        print("Keine Starts gemessen.")

    print(f"Ø Queue (Jahr): {np.mean(queue_weekly):.1f} | Max Queue: {np.max(queue_weekly):.0f}")
    if tip is None:
        print(f"Kipppunkt: keiner (Queue nicht {CONSEC_WEEKS_FOR_TIP} Wochen am Stück > 0)")
    else:
        print(f"Kipppunkt: ab Woche {tip} (Queue > 0 für {CONSEC_WEEKS_FOR_TIP} Wochen in Folge)")

    max_starts_week = PLACES / THERAPY_WEEKS
    print(f"Faustformel: max Starts/Woche ≈ {max_starts_week:,.0f}")

if __name__ == "__main__":
    run()

=== VT (vereinfacht) ===
Theo Plätze: 116,100 | Eff: 0.727 -> Plätze effektiv: 84,396
Therapie: 24 Wochen | Gate P_START: 0.888
Nachfrage: lambda0=6000.0, growth=0.008/Woche (exponentiell)
Arrivals: 387,633 | Accepted: 344,359 | Dropped: 43,274
Ø Wartezeit: 16.62 Wochen | Median: 12.45 | 95%: 35.68 | Max: 37.23
Ø Queue (Jahr): 50016.4 | Max Queue: 148723
Kipppunkt: ab Woche 16 (Queue > 0 für 3 Wochen in Folge)
Faustformel: max Starts/Woche ≈ 3,516


In [11]:
import simpy
import numpy as np

# ============================================================
# GRUNDIDEE (kurz)
# ============================================================
# - Kapazität = Therapieplätze (parallele Therapien) aus Teilzeit/Vollzeit
# - Patient*innen kommen zufällig (Poisson-Prozess) und Nachfrage wächst exponentiell
# - Gate: nur Anteil "Ja" bleibt im System
# - Start nur bei freiem Platz (Warteschlange FIFO)
# - Therapie blockiert Platz für "effektive Dauer" (z.B. 24 oder 30 Wochen)
# - Effizienz kann modelliert werden als:
#   A) weniger Plätze (capacity *= eff) oder
#   B) längere Therapiedauer (therapy_weeks_eff > 24)
# - Reverse Engineering: berechnet Effizienz-Faktor, damit Angebot≈Nachfrage (Sättigung)
# - Kipppunkt: erste Woche, ab der Queue mehrere Wochen am Stück > 0

# ============================================================
# 1) INPUTS: VT DATEN
# ============================================================
WEEKS_PER_YEAR = 52
SEED = 42

# Teilzeit / Vollzeit VT
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510
PARALLEL_CAP_TEILZEIT = 9
PARALLEL_CAP_VOLLZEIT = 19

THEO_PLACES = TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
# = 116_100

# ============================================================
# 2) THERAPIE & URLAUBS-EFFEKT
# ============================================================
THERAPY_WEEKS_BASE = 24  # 24 Sitzungen ~ 24 Wochen bei 1/Woche

# Modellwahl: Wie soll "Urlaub/Krankheit/Feiertage" abgebildet werden?
# - "capacity": weniger effektive Plätze (eff_factor wirkt auf Plätze)
# - "duration": Therapie dauert länger (z.B. 30 Wochen statt 24)
# - "both": beides gleichzeitig (für "pessimistisches" Szenario)
EFFICIENCY_MODE = "duration"  # "capacity" | "duration" | "both"

# A) Effizienz als Kapazitätsreduktion (Plätze)
OFF_WEEKS = 10          # z.B. 6 Urlaub + 2 Krankheit/Fortbildung + 2 Feiertage
FACTOR_OTHER = 0.90     # Admin/No-shows etc.
EFF_CAPACITY = ((WEEKS_PER_YEAR - OFF_WEEKS) / WEEKS_PER_YEAR) * FACTOR_OTHER
# Beispiel: 42/52 * 0.90 ≈ 0.727

# B) Effizienz als Therapiedauer-Verlängerung (Urlaubs-Effekt)
# Beispiel aus deinem Screenshot: 24 -> ~30 Wochen
THERAPY_WEEKS_EFFECTIVE = 30  # <-- hier spielen (24, 28, 30, 32...)

# ============================================================
# 3) NACHFRAGE (Gate) + ANKÜNFTE (Exponentiell)
# ============================================================
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR  # ~0.8882

# Ankunftsrate pro Woche: lambda(t) = lambda0 * exp(g*t)
LAMBDA0 = 6000.0            # Startniveau (pro Woche)
GROWTH = 0.008              # Wachstum pro Woche (0.008 ~ ca. +50% pro Jahr)

def lambda_t(t_week: float) -> float:
    return LAMBDA0 * np.exp(GROWTH * t_week)

# Optionaler Start-Delay (z.B. Diagnostik/Antrag)
START_DELAY_MODE = "uniform"  # "uniform" | "exponential" | "none"
MAX_DELAY_WEEKS = 2
MEAN_DELAY_WEEKS = 1.0

def sample_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_DELAY_WEEKS + 1))
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(MEAN_DELAY_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# ============================================================
# 4) REVERSE ENGINEERING (SÄTTIGUNG / FAKTOR)
# ============================================================
# Idee aus deinem Screenshot:
# Leistung pro Slot pro Jahr = 52 / TherapieDauerEffektiv
# Benötigte Slots = NachfrageJa / LeistungProSlot
# Faktor = benötigteSlots / theoretischeSlots
#
# ABER: das ist eine "Jahres-Bilanz". In der Simulation kann es wegen Wachstum/Randomness früher kippen.
# Trotzdem ist es super für eine erklärbare Folie.

def reverse_engineer_factor(yes_patients_year: int, theo_places: int, therapy_weeks_eff: float) -> float:
    throughput_per_place_year = WEEKS_PER_YEAR / therapy_weeks_eff  # Patient*innen pro Platz/Jahr
    needed_places = yes_patients_year / throughput_per_place_year
    factor = needed_places / theo_places
    return float(factor)

# Optional: Wenn du willst, dass der Code den Faktor automatisch setzt:
AUTO_SET_EFF_FROM_REVERSE = False  # True = Faktor wird aus Reverse-Engineering berechnet

# Wenn AUTO_SET_EFF_FROM_REVERSE True ist:
# - Bei MODE="capacity": EFF_CAPACITY wird automatisch gesetzt
# - Bei MODE="both": EFF_CAPACITY wird gesetzt, THERAPY_WEEKS_EFFECTIVE bleibt wie angegeben
# - Bei MODE="duration": ergibt "Kapazitäts-Faktor" weniger Sinn, weil wir über Dauer arbeiten

# ============================================================
# 5) KIPPPUNKT-REGEL
# ============================================================
# Kipppunkt = erste Woche, ab der Queue > 0 für X Wochen am Stück
CONSEC_WEEKS_FOR_TIP = 3

def find_tipping_week(queue_weekly: np.ndarray, consec: int) -> int | None:
    run = 0
    for w, q in enumerate(queue_weekly):
        run = run + 1 if q > 0 else 0
        if run >= consec:
            return w - consec + 1
    return None

# ============================================================
# 6) SIMULATION
# ============================================================
def effective_places(theo_places: int, mode: str) -> int:
    if mode == "capacity":
        return max(1, int(round(theo_places * EFF_CAPACITY)))
    if mode == "duration":
        return theo_places
    if mode == "both":
        return max(1, int(round(theo_places * EFF_CAPACITY)))
    raise ValueError("EFFICIENCY_MODE muss 'capacity', 'duration' oder 'both' sein")

def effective_therapy_weeks(base_weeks: int, mode: str) -> int:
    if mode == "capacity":
        return base_weeks
    if mode == "duration":
        return int(THERAPY_WEEKS_EFFECTIVE)
    if mode == "both":
        return int(THERAPY_WEEKS_EFFECTIVE)
    raise ValueError("EFFICIENCY_MODE muss 'capacity', 'duration' oder 'both' sein")

def run_sim():
    global EFF_CAPACITY

    # Reverse Engineering optional anwenden (für Folien/Szenario-Kalibrierung)
    rev_factor = reverse_engineer_factor(YES_PATIENTS_YEAR, THEO_PLACES, THERAPY_WEEKS_EFFECTIVE)
    if AUTO_SET_EFF_FROM_REVERSE and EFFICIENCY_MODE in ("capacity", "both"):
        EFF_CAPACITY = rev_factor  # setzt Kapazitätsfaktor so, dass Jahresbilanz ~ Sättigung

    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    PLACES = effective_places(THEO_PLACES, EFFICIENCY_MODE)
    THERAPY_WEEKS = effective_therapy_weeks(THERAPY_WEEKS_BASE, EFFICIENCY_MODE)

    slots = simpy.Resource(env, capacity=PLACES)

    # Ergebnislisten
    wait_times = []
    queue_weekly = []  # einmal pro Woche
    arrived_total = 0
    accepted_total = 0
    dropped_total = 0

    def patient():
        nonlocal accepted_total, dropped_total
        t_arr = env.now

        # Gate
        if rng.random() > P_START:
            dropped_total += 1
            return
        accepted_total += 1

        # optionaler Delay
        d = sample_delay(rng)
        if d > 0:
            yield env.timeout(d)

        # Warten auf freien Platz
        with slots.request() as req:
            yield req
            wait_times.append(env.now - t_arr)
            yield env.timeout(THERAPY_WEEKS)

    def arrivals_continuous():
        nonlocal arrived_total
        while env.now < WEEKS_PER_YEAR:
            lam = lambda_t(env.now)
            inter = rng.exponential(1.0 / lam)
            yield env.timeout(inter)
            if env.now >= WEEKS_PER_YEAR:
                break
            arrived_total += 1
            env.process(patient())

    def monitor_weekly():
        for _ in range(WEEKS_PER_YEAR):
            queue_weekly.append(len(slots.queue))
            yield env.timeout(1.0)

    env.process(arrivals_continuous())
    env.process(monitor_weekly())

    # Jahr + Puffer, damit Therapien auslaufen
    env.run(until=WEEKS_PER_YEAR + THERAPY_WEEKS + 10)

    # Auswertung
    wt = np.array(wait_times) if wait_times else np.array([])
    queue_weekly_arr = np.array(queue_weekly) if queue_weekly else np.array([])

    tipping = find_tipping_week(queue_weekly_arr, CONSEC_WEEKS_FOR_TIP)

    # Reporting
    print("=== VT Simulation (ausführlich + Effizienz + Exponential + Kipppunkt) ===")
    print(f"EFFICIENCY_MODE: {EFFICIENCY_MODE}")
    print(f"Theoretische Plätze: {THEO_PLACES:,}")

    if EFFICIENCY_MODE in ("capacity", "both"):
        print(f"Kapazitäts-Effizienz (EFF_CAPACITY): {EFF_CAPACITY:.3f}  (OFF_WEEKS={OFF_WEEKS}, FACTOR_OTHER={FACTOR_OTHER})")
    else:
        print("Kapazitäts-Effizienz: (nicht verwendet, weil MODE='duration')")

    if EFFICIENCY_MODE in ("duration", "both"):
        print(f"Therapie-Dauer effektiv: {THERAPY_WEEKS} Wochen (statt {THERAPY_WEEKS_BASE})")
    else:
        print(f"Therapie-Dauer: {THERAPY_WEEKS} Wochen")

    print(f"Effektive Plätze: {PLACES:,}")
    print(f"Gate P_START: {P_START:.4f}")
    print(f"Ankünfte: lambda(t)=LAMBDA0*exp(g*t), LAMBDA0={LAMBDA0}, g={GROWTH}/Woche")
    print(f"Start-Delay: {START_DELAY_MODE}")

    print("\n--- Nachfrage ---")
    print(f"Arrivals (vor Gate): {arrived_total:,}")
    print(f"Accepted (nach Gate): {accepted_total:,}")
    print(f"Dropped: {dropped_total:,} ({(dropped_total/arrived_total*100 if arrived_total else 0):.1f}%)")

    if wt.size:
        print("\n--- Wartezeiten bis Therapiebeginn (inkl. Delay) ---")
        print(f"Starts gemessen: {wt.size:,}")
        print(f"Ø Wartezeit: {wt.mean():.2f} Wochen")
        print(f"Median: {np.quantile(wt, 0.50):.2f}")
        print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
        print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
        print(f"Max: {wt.max():.2f}")
    else:
        print("\nKeine Starts gemessen (sehr ungewöhnlich bei diesen Parametern).")

    if queue_weekly_arr.size:
        print("\n--- Warteschlange (wöchentlich) ---")
        print(f"Ø Queue-Länge: {queue_weekly_arr.mean():.1f}")
        print(f"Max Queue-Länge: {queue_weekly_arr.max():.0f}")

    print("\n--- Kipppunkt ---")
    if tipping is None:
        print(f"Kein Kipppunkt (Queue nicht {CONSEC_WEEKS_FOR_TIP} Wochen am Stück > 0).")
    else:
        print(f"Kipppunkt ab Woche {tipping} (Queue > 0 für {CONSEC_WEEKS_FOR_TIP} Wochen in Folge).")

    print("\n--- Reverse Engineering (für Folien) ---")
    print(f"Wenn Therapie effektiv {THERAPY_WEEKS_EFFECTIVE} Wochen dauert:")
    print(f"Benötigter Faktor (needed/theoretical) ≈ {rev_factor:.2f}  (= {rev_factor*100:.0f}%)")
    print("Interpretation: so viel 'effektive Slots/Plätze' braucht man relativ zur Theorie, um Jahresnachfrage zu bedienen.")

    # Faustformel (mit effektiven Werten)
    max_starts_week = PLACES / THERAPY_WEEKS
    print("\n--- Faustformel (Simulationseinstellungen) ---")
    print(f"Max Starts/Woche ≈ {max_starts_week:,.0f}")
    print("Wenn (Accepted Arrivals/Woche) langfristig darüber liegt -> Warteschlange wächst.")

if __name__ == "__main__":
    run_sim()

=== VT Simulation (ausführlich + Effizienz + Exponential + Kipppunkt) ===
EFFICIENCY_MODE: duration
Theoretische Plätze: 116,100
Kapazitäts-Effizienz: (nicht verwendet, weil MODE='duration')
Therapie-Dauer effektiv: 30 Wochen (statt 24)
Effektive Plätze: 116,100
Gate P_START: 0.8882
Ankünfte: lambda(t)=LAMBDA0*exp(g*t), LAMBDA0=6000.0, g=0.008/Woche
Start-Delay: uniform

--- Nachfrage ---
Arrivals (vor Gate): 387,633
Accepted (nach Gate): 344,359
Dropped: 43,274 (11.2%)

--- Wartezeiten bis Therapiebeginn (inkl. Delay) ---
Starts gemessen: 344,359
Ø Wartezeit: 13.05 Wochen
Median: 12.41
90% Quantil: 27.06
95% Quantil: 27.99
Max: 29.09

--- Warteschlange (wöchentlich) ---
Ø Queue-Länge: 38080.0
Max Queue-Länge: 96042

--- Kipppunkt ---
Kipppunkt ab Woche 21 (Queue > 0 für 3 Wochen in Folge).

--- Reverse Engineering (für Folien) ---
Wenn Therapie effektiv 30 Wochen dauert:
Benötigter Faktor (needed/theoretical) ≈ 0.42  (= 42%)
Interpretation: so viel 'effektive Slots/Plätze' braucht man

In [13]:
import simpy
import numpy as np

# ============================================================
# 1) ZEITRAUM: 3 JAHRE
# ============================================================
YEARS = 3
WEEKS_PER_YEAR = 52
TOTAL_WEEKS = YEARS * WEEKS_PER_YEAR

SEED = 42

# ============================================================
# 2) KAPAZITÄT: Therapieplätze aus Teilzeit/Vollzeit
# ============================================================
TEILZEIT_THERAPEUTEN = 5490
VOLLZEIT_THERAPEUTEN = 3510
PARALLEL_CAP_TEILZEIT = 9
PARALLEL_CAP_VOLLZEIT = 19

THEO_PLACES = TEILZEIT_THERAPEUTEN * PARALLEL_CAP_TEILZEIT + VOLLZEIT_THERAPEUTEN * PARALLEL_CAP_VOLLZEIT
# 116_100

# ============================================================
# 3) THERAPIE & URLAUBS-EFFEKT (Effizienz)
# ============================================================
THERAPY_WEEKS_BASE = 24

EFFICIENCY_MODE = "duration"  # "capacity" | "duration" | "both"

# A) Kapazität reduzieren (Plätze * Faktor)
OFF_WEEKS = 10
FACTOR_OTHER = 0.90
EFF_CAPACITY = ((WEEKS_PER_YEAR - OFF_WEEKS) / WEEKS_PER_YEAR) * FACTOR_OTHER

# B) Dauer verlängern (z.B. 24 -> 30 Wochen)
THERAPY_WEEKS_EFFECTIVE = 30  # <-- hier spielen

def effective_places(theo_places: int, mode: str) -> int:
    if mode == "capacity":
        return max(1, int(round(theo_places * EFF_CAPACITY)))
    if mode == "duration":
        return theo_places
    if mode == "both":
        return max(1, int(round(theo_places * EFF_CAPACITY)))
    raise ValueError("EFFICIENCY_MODE muss 'capacity', 'duration' oder 'both' sein")

def effective_therapy_weeks(base_weeks: int, mode: str) -> int:
    if mode == "capacity":
        return base_weeks
    if mode == "duration":
        return int(THERAPY_WEEKS_EFFECTIVE)
    if mode == "both":
        return int(THERAPY_WEEKS_EFFECTIVE)
    raise ValueError("EFFICIENCY_MODE muss 'capacity', 'duration' oder 'both' sein")

# ============================================================
# 4) NACHFRAGE: Gate + exponentielles Wachstum
# ============================================================
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR  # ~0.8882

# lambda(t) = LAMBDA0 * exp(g * t)
LAMBDA0 = 4200.0
GROWTH = 0.008

def lambda_t(t_week: float) -> float:
    return LAMBDA0 * np.exp(GROWTH * t_week)

# Optionaler Start-Delay
START_DELAY_MODE = "uniform"  # "uniform" | "exponential" | "none"
MAX_DELAY_WEEKS = 2
MEAN_DELAY_WEEKS = 1.0

def sample_delay(rng: np.random.Generator) -> float:
    if START_DELAY_MODE == "none":
        return 0.0
    if START_DELAY_MODE == "uniform":
        return float(rng.integers(0, MAX_DELAY_WEEKS + 1))
    if START_DELAY_MODE == "exponential":
        return float(rng.exponential(MEAN_DELAY_WEEKS))
    raise ValueError("START_DELAY_MODE muss 'uniform', 'exponential' oder 'none' sein")

# ============================================================
# 5) KIPPPUNKT
# ============================================================
CONSEC_WEEKS_FOR_TIP = 3

def find_tipping_week(queue_weekly: np.ndarray, consec: int) -> int | None:
    run = 0
    for w, q in enumerate(queue_weekly):
        run = run + 1 if q > 0 else 0
        if run >= consec:
            return w - consec + 1
    return None

# ============================================================
# 6) REVERSE ENGINEERING (für Folien)
# ============================================================
def reverse_engineer_factor(yes_patients_year: int, theo_places: int, therapy_weeks_eff: float) -> float:
    throughput_per_place_year = WEEKS_PER_YEAR / therapy_weeks_eff
    needed_places = yes_patients_year / throughput_per_place_year
    return float(needed_places / theo_places)

# ============================================================
# 7) SIMULATION
# ============================================================
def run_sim():
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()

    PLACES = effective_places(THEO_PLACES, EFFICIENCY_MODE)
    THERAPY_WEEKS = effective_therapy_weeks(THERAPY_WEEKS_BASE, EFFICIENCY_MODE)

    slots = simpy.Resource(env, capacity=PLACES)

    wait_times = []
    queue_weekly = []  # Queue-Länge für alle Wochen (3 Jahre)
    arrived_total = 0
    accepted_total = 0
    dropped_total = 0

    def patient():
        nonlocal accepted_total, dropped_total
        t_arr = env.now

        if rng.random() > P_START:
            dropped_total += 1
            return
        accepted_total += 1

        d = sample_delay(rng)
        if d > 0:
            yield env.timeout(d)

        with slots.request() as req:
            yield req
            wait_times.append(env.now - t_arr)
            yield env.timeout(THERAPY_WEEKS)

    def arrivals_continuous():
        nonlocal arrived_total
        while env.now < TOTAL_WEEKS:
            lam = lambda_t(env.now)
            inter = rng.exponential(1.0 / lam)
            yield env.timeout(inter)

            if env.now >= TOTAL_WEEKS:
                break

            arrived_total += 1
            env.process(patient())

    def monitor_weekly():
        for _ in range(TOTAL_WEEKS):
            queue_weekly.append(len(slots.queue))
            yield env.timeout(1.0)

    env.process(arrivals_continuous())
    env.process(monitor_weekly())

    # 3 Jahre + Puffer, damit Therapien auslaufen
    env.run(until=TOTAL_WEEKS + THERAPY_WEEKS + 10)

    wt = np.array(wait_times) if wait_times else np.array([])
    q = np.array(queue_weekly) if queue_weekly else np.array([])

    tipping = find_tipping_week(q, CONSEC_WEEKS_FOR_TIP)

    print("=== VT Simulation (3 Jahre) ===")
    print(f"Zeitraum: {YEARS} Jahre = {TOTAL_WEEKS} Wochen")
    print(f"EFFICIENCY_MODE: {EFFICIENCY_MODE}")
    print(f"Theoretische Plätze: {THEO_PLACES:,}")
    print(f"Effektive Plätze: {PLACES:,}")
    print(f"Therapie-Dauer effektiv: {THERAPY_WEEKS} Wochen")
    print(f"Gate P_START: {P_START:.4f}")
    print(f"Ankünfte: lambda(t)=LAMBDA0*exp(g*t), LAMBDA0={LAMBDA0}, g={GROWTH}/Woche")
    print(f"Start-Delay: {START_DELAY_MODE}")

    print("\n--- Nachfrage ---")
    print(f"Arrivals (vor Gate): {arrived_total:,}")
    print(f"Accepted (nach Gate): {accepted_total:,}")
    print(f"Dropped: {dropped_total:,} ({(dropped_total/arrived_total*100 if arrived_total else 0):.1f}%)")

    if wt.size:
        print("\n--- Wartezeiten bis Therapiebeginn (inkl. Delay) ---")
        print(f"Starts gemessen: {wt.size:,}")
        print(f"Ø Wartezeit: {wt.mean():.2f} Wochen")
        print(f"Median: {np.quantile(wt, 0.50):.2f}")
        print(f"90% Quantil: {np.quantile(wt, 0.90):.2f}")
        print(f"95% Quantil: {np.quantile(wt, 0.95):.2f}")
        print(f"Max: {wt.max():.2f}")

    if q.size:
        print("\n--- Warteschlange (wöchentlich, über 3 Jahre) ---")
        print(f"Ø Queue-Länge: {q.mean():.1f}")
        print(f"Max Queue-Länge: {q.max():.0f}")

    print("\n--- Kipppunkt ---")
    if tipping is None:
        print(f"Kein Kipppunkt (Queue nicht {CONSEC_WEEKS_FOR_TIP} Wochen am Stück > 0).")
    else:
        year = tipping // WEEKS_PER_YEAR + 1
        week_in_year = tipping % WEEKS_PER_YEAR
        print(f"Kipppunkt ab Woche {tipping} (Jahr {year}, KW~{week_in_year})")

    print("\n--- Reverse Engineering (für Folien) ---")
    rev_factor = reverse_engineer_factor(YES_PATIENTS_YEAR, THEO_PLACES, THERAPY_WEEKS_EFFECTIVE)
    print(f"Bei Therapie-Dauer {THERAPY_WEEKS_EFFECTIVE} Wochen: Faktor ≈ {rev_factor:.2f} ({rev_factor*100:.0f}%)")

    max_starts_week = PLACES / THERAPY_WEEKS
    print("\n--- Faustformel ---")
    print(f"Max Starts/Woche ≈ {max_starts_week:,.0f}")

if __name__ == "__main__":
    run_sim()

=== VT Simulation (3 Jahre) ===
Zeitraum: 3 Jahre = 156 Wochen
EFFICIENCY_MODE: duration
Theoretische Plätze: 116,100
Effektive Plätze: 116,100
Therapie-Dauer effektiv: 30 Wochen
Gate P_START: 0.8882
Ankünfte: lambda(t)=LAMBDA0*exp(g*t), LAMBDA0=4200.0, g=0.008/Woche
Start-Delay: uniform

--- Nachfrage ---
Arrivals (vor Gate): 1,305,128
Accepted (nach Gate): 1,159,310
Dropped: 145,818 (11.2%)

--- Wartezeiten bis Therapiebeginn (inkl. Delay) ---
Starts gemessen: 756,089
Ø Wartezeit: 28.45 Wochen
Median: 24.58
90% Quantil: 62.82
95% Quantil: 70.23
Max: 76.65

--- Warteschlange (wöchentlich, über 3 Jahre) ---
Ø Queue-Länge: 153034.0
Max Queue-Länge: 537954

--- Kipppunkt ---
Kipppunkt ab Woche 29 (Jahr 1, KW~29)

--- Reverse Engineering (für Folien) ---
Bei Therapie-Dauer 30 Wochen: Faktor ≈ 0.42 (42%)

--- Faustformel ---
Max Starts/Woche ≈ 3,870


In [15]:
import simpy
import numpy as np

# -----------------------------
# KONFIGURATION
# -----------------------------
WEEKS_PER_YEAR = 52
SIMULATION_DURATION = int(WEEKS_PER_YEAR * 3) # 3 Jahre Simulation
SEED = 42

# 1. NACHFRAGE
TOTAL_PATIENTS_YEAR = 95201
YES_PATIENTS_YEAR = 84557
P_START_THERAPY = YES_PATIENTS_YEAR / TOTAL_PATIENTS_YEAR 

# 2. KAPAZITÄT
SLOTS_PART_TIME = 5490 * 9
SLOTS_FULL_TIME = 3510 * 19
THEORETICAL_CAPACITY = SLOTS_PART_TIME + SLOTS_FULL_TIME 

# 3. ZEIT-PARAMETER
SESSIONS_REQUIRED = 24  
WORKING_WEEKS = 42  # Abzug von Urlaub/Krankheit
TIME_STRETCH = 52 / WORKING_WEEKS 
REAL_DURATION_WEEKS = SESSIONS_REQUIRED * TIME_STRETCH # ca. 30 Wochen

# ANPASSUNG (KORRIGIERT):
# Mit der korrigierten Zeitschleife (siehe unten) ist das System nun dauerhaft belastet.
# Ein Faktor von 0.40 (40%) sorgt für eine Auslastung nah an 100%, 
# was zu realistischen Staus führt, ohne dass die Wartezeit ins Unendliche wächst.
# Benötigte Plätze (Sättigung) = Starts/Jahr * Dauer / Wochen/Jahr
NEEDED_SLOTS = YES_PATIENTS_YEAR * (REAL_DURATION_WEEKS / WEEKS_PER_YEAR)

# Effizienzfaktor ausrechnen
EFFICIENCY_FACTOR = NEEDED_SLOTS / THEORETICAL_CAPACITY
REAL_CAPACITY = int(THEORETICAL_CAPACITY * EFFICIENCY_FACTOR)

# Wir starten mit voller Last
INITIAL_WEEKLY_RATE = (TOTAL_PATIENTS_YEAR / WEEKS_PER_YEAR) * 1.0
GROWTH_PER_WEEK = 0.005 

stats_wait_times = []

# -----------------------------
# PROZESSLOGIK
# -----------------------------

def patient_process(env, name, therapy_slots, dropped_counter):
    t_ankunft = env.now
    if np.random.random() > P_START_THERAPY:
        dropped_counter['count'] += 1
        return 

    with therapy_slots.request() as req:
        yield req 
        stats_wait_times.append(env.now - t_ankunft)
        yield env.timeout(REAL_DURATION_WEEKS)

def pre_fill_slot(env, therapy_slots, rng):
    with therapy_slots.request() as req:
        yield req
        yield env.timeout(rng.uniform(0, REAL_DURATION_WEEKS))

def setup(env, capacity, rng):
    therapy_slots = simpy.Resource(env, capacity=capacity)
    
    # Warm-up
    # Startet nicht leer 
    # Zufällig -> Restdauer Therapie 
    for _ in range(capacity):
        env.process(pre_fill_slot(env, therapy_slots, rng))

    # Patien die gar nicht starten     
    dropped_counter = {'count': 0}
    i = 0
    current_rate = INITIAL_WEEKLY_RATE

    # KORREKTUR: Generator läuft nun über die volle Simulationsdauer!
    # 3 Jahre 
    # (Vorher stand hier WEEKS_PER_YEAR, was falsch war)
    while env.now < SIMULATION_DURATION:
        scale = 1.0 / current_rate
        interarrival_time = rng.exponential(scale=scale)
        yield env.timeout(interarrival_time)
        
        env.process(patient_process(env, f"Patient_{i}", therapy_slots, dropped_counter))
        i += 1
        current_rate = current_rate * (1 + (GROWTH_PER_WEEK * interarrival_time))

# -----------------------------
# MAIN & MEDIAN-ANALYSE
# -----------------------------
if __name__ == "__main__":
    print(f"--- SIMULATION (MIT MEDIAN & KORRIGIERTEM LOOP) ---")
    print(f"Laufzeit: {SIMULATION_DURATION} Wochen")
    print(f"Effizienz-Faktor: {EFFICIENCY_FACTOR}")
    print(f"Kapazität: {REAL_CAPACITY} Slots")
    
    rng = np.random.default_rng(SEED)
    env = simpy.Environment()
    env.process(setup(env, capacity=REAL_CAPACITY, rng=rng))
    env.run(until=SIMULATION_DURATION)

    waits = np.array(stats_wait_times)
    n_started = len(waits)

    print(f"\n" + "="*60)
    print(f"       STATISTIK (MEDIAN VS. MITTELWERT)       ")
    print(f"="*60)
    
    if n_started > 0:
        # Mittelwert
        mean_weeks = np.mean(waits)
        mean_days = mean_weeks * 7
        
        # Median (50-Prozent-Hürde)
        median_weeks = np.median(waits)
        median_days = median_weeks * 7
        
        # P90 (Risiko)
        p90_weeks = np.quantile(waits, 0.90)
        p90_days = p90_weeks * 7

        print(f"Anzahl Patienten: {n_started}")
        print(f"-"*60)
        # Hier sehen Sie den Unterschied direkt:
        print(f"MITTELWERT (Mean):       {mean_days:6.1f} Tage")
        print(f"MEDIAN (Typischer Fall): {median_days:6.1f} Tage")
        print(f"-"*60)
        print(f"90% warten kürzer als:   {p90_days:6.1f} Tage")
        print(f"Maximale Wartezeit:      {np.max(waits)*7:6.1f} Tage")
        
    else:
        print("Keine Daten.")
        

--- SIMULATION (MIT MEDIAN & KORRIGIERTEM LOOP) ---
Laufzeit: 156 Wochen
Effizienz-Faktor: 0.4161781715270087
Kapazität: 48318 Slots

       STATISTIK (MEDIAN VS. MITTELWERT)       
Anzahl Patienten: 253692
------------------------------------------------------------
MITTELWERT (Mean):        103.9 Tage
MEDIAN (Typischer Fall):   85.6 Tage
------------------------------------------------------------
90% warten kürzer als:    239.5 Tage
Maximale Wartezeit:       285.5 Tage
